In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# !pip install nltk
# !pip install spacy
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
import re
import string
from nltk.corpus import stopwords, brown,wordnet
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk import pos_tag,RegexpParser
from nltk.stem import WordNetLemmatizer,SnowballStemmer
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
from IPython.display import Image,display
# !pip install pdfplumber
import pdfplumber

#  **1. Data *extraction*** <br/>
Read the documents : 

In [3]:
# f = open("data/PMBOK 5th.pdf", "rt", errors="ignore")
# print(f.read()[:1000])
# f.close()

In [4]:
# #!pip install pdfplumber
# import pdfplumber
# with pdfplumber.open('data/PMBOK 5th.pdf') as pdf:
#     first_page = pdf.pages[2]
#     print(first_page.extract_text())
# pdf.close()

In [5]:
# for i in range(334,338):
#     with pdfplumber.open("data/PMBOK 5th.pdf") as pdf:
#         sentence = str(pdf.pages[i].extract_text())
#         print("page "+str(i)+" : "+sentence[-500:])

Extract useful information : 

In [6]:
# L = []
# x = 0
# for i in range(334,379,2):
#     L.append(list([i,i+1]))
# for j in range(len(L)):
#     with pdfplumber.open("data/PMBOK 5th.pdf") as pdf:
#         page1 = str(pdf.pages[L[j][0]].extract_text())
#         page2 = str(pdf.pages[L[j][1]].extract_text())
#         if (page1.find("©2013 Project Management Institute. A Guide to the Project Management Body of Knowledge (PMBOK® Guide) – Fifth Edition  "+str(L[j][0]-25)+"\n")!=-1) and (page2.find(str(L[j][1]-25)+" ©2013 Project Management Institute. A Guide to the Project Management Body of Knowledge (PMBOK® Guide) – Fifth Edition \n")!=-1):
#             x+=1
#         else:
#             print('problem!!')

In [7]:
# msg = ''
# for i in range(334,380):
#     with pdfplumber.open("data/PMBOK 5th.pdf") as pdf:
#         sentence = str(pdf.pages[i].extract_text())
#         if sentence.find("Licensed To: Jorge Diego Fuentes Sanchez PMI MemberID: 2399412\nThis copy is a PMI Member benefit, not for distribution, sale, or reproduction.")==-1:
#             msg+='problem'
# print(msg)

In [8]:
# L = list()
# count = 0
# for i in range(334,380):
#     with pdfplumber.open("data/PMBOK 5th.pdf") as pdf:
#         sentence = str(pdf.pages[i].extract_text())
#         index = sentence.find("Licensed To: Jorge Diego Fuentes Sanchez PMI MemberID: 2399412\nThis copy is a PMI Member benefit, not for distribution, sale, or reproduction.")
#         L.append(sentence[index-11:index])
#         count+=1
#         if count>10:
#             break

In [9]:
# L

In [3]:
dictionary = {'the whole page':[337,345],'2 images on the buttom':[344,338,353,367,374],'table on the top':[343], 'one image on the buttom':[356,361,362,363,364,365],
                  'one image on the top':[351],'2 images on the top':[359],'one image on the middle':[342]}

In [4]:
start = 334
end = 379
text = ''
with pdfplumber.open('data/PMBOK 5th.pdf') as pdf:
    for i in range(start,end+1):
        if i in [356,361,362,363,364,365] :
            text = str(pdf.pages[i].extract_text())
            reg = re.compile(r'((objectives)|(\(Figure 11-16\))|(values)|(design)|(analysis)|(project))\.\n')
            matches = reg.finditer(text)
            for match in matches:
                print(match)

<re.Match object; span=(1440, 1449), match='project.\n'>
<re.Match object; span=(1184, 1194), match='analysis.\n'>
<re.Match object; span=(791, 799), match='design.\n'>
<re.Match object; span=(1529, 1537), match='values.\n'>
<re.Match object; span=(689, 705), match='(Figure 11-16).\n'>
<re.Match object; span=(993, 1005), match='objectives.\n'>


In [5]:
def extract_useful_text(dictionary):
    start = 334
    end = 379
    text = ''
    with pdfplumber.open('data/PMBOK 5th.pdf') as pdf:
        for i in range(start,end+1):
            if i in dictionary['the whole page']:
                new_page = ''
            elif i in dictionary['2 images on the buttom']:
                indice = int(str(pdf.pages[i].extract_text()).find('process.\n'))
                new_page = str(pdf.pages[i].extract_text())[:indice+len('process.\n')]
                print(str(i)+':\n'+new_page)
            elif i in dictionary['table on the top']:
                indice = int(str(pdf.pages[i].extract_text()).find('•'))
                new_page = str(pdf.pages[i].extract_text())[indice:]
                print(str(i)+':\n'+new_page)
            elif i in dictionary['one image on the buttom']:
                reg = re.compile(r'((objectives)|(\(Figure 11-16\))|(values)|(design)|(analysis)|(project))\.\n')
                matches = reg.finditer(str(pdf.pages[i].extract_text()))
                for match in matches:
                    indice = match.span()[1]
                    new_page = str(pdf.pages[i].extract_text())[:indice]
                print(str(i)+':\n'+new_page)
            elif i in dictionary['one image on the top']:
                indice = int(str(pdf.pages[i].extract_text()).find('11.2.2.6 SWot Analysis'))
                new_page = str(pdf.pages[i].extract_text())[indice:]
                print(str(i)+':\n'+new_page)
            elif i in dictionary['2 images on the top']:
                indice = int(str(pdf.pages[i].extract_text()).find('Perform Quantitative Risk Analysis is performed on risks'))
                new_page = str(pdf.pages[i].extract_text())[indice:]
                print(str(i)+':\n'+new_page)
            else:
                new_page = str(pdf.pages[i].extract_text())
            text+=new_page
    return text

In [6]:
df1 = extract_useful_text(dictionary)

338:
11 - PROJECT RISK MANAGEMENT
11.1 Plan risk Management
Plan Risk Management is the process of defining how to conduct risk management activities for a project. The 
key benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate 
with both the risks and the importance of the project to the organization. The risk management plan is vital to 
communicate with and obtain agreement and support from all stakeholders to ensure the risk management process 
is supported and performed effectively over the project life cycle. The inputs, tools and techniques, and outputs of 
this process are depicted in Figure 11-2. Figure 11-3 depicts the data flow diagram of the process.

343:
•	   Probability and impact matrix. A probability and impact matrix is a grid for mapping the probability 
of each risk occurrence and its impact on project objectives if that risk occurs. Risks are prioritized 
according to their potential implications for having 

363:
11 - PROJECT RISK MANAGEMENT
11.4.2.2 Quantitative risk Analysis and Modeling techniques
Commonly used techniques use both event-oriented and project-oriented analysis approaches, including:
•	  Sensitivity analysis. Sensitivity analysis helps to determine which risks have the most potential 
impact on the project. It helps to understand how the variations in project’s objectives correlate with 
variations in different uncertainties. Conversely, it examines the extent to which the uncertainty of 
each project element affects the objective being studied when all other uncertain elements are held at 
their baseline values. One typical display of sensitivity analysis is the tornado diagram (Figure 11-15), 
which is useful for comparing relative importance and impact of variables that have a high degree of 
uncertainty to those that are more stable. The Tornado diagram is also helpful in analyzing risk-taking 
scenarios enabled on specific risks whose quantitative analysis highlights 

In [7]:
df1.find('Work performance data related to various performance results possibly impacted \nby risks includes, but is not limited to:',75114)

-1

In [8]:
df1[4736:4836]

'•\t  Risk tolerance, which is the degree, amount, or volume of risk that an organization or individua'

In [9]:
df1.find('\n11.4.2')

47506

In [10]:
df1[41640:]

'th Edition \nLicensed To: Jorge Diego Fuentes Sanchez PMI MemberID: 2399412\nThis copy is a PMI Member benefit, not for distribution, sale, or reproduction.11 - PROJECT RISK MANAGEMENT\n11.3.2.5 risk urgency Assessment\nRisks requiring near-term responses may be considered more urgent to address. Indicators of priority may \ninclude probability of detecting the risk, time to affect a risk response, symptoms and warning signs, and the \nrisk rating. In some qualitative analyses, the assessment of risk urgency is combined with the risk ranking that is \ndetermined from the probability and impact matrix to give a final risk severity rating.\n11.3.2.6 Expert Judgment\nExpert judgment is required to assess the probability and impact of each risk to determine its location in \nthe matrix shown in Figure 11-10. Experts generally are those having experience with similar, recent projects. \nGathering expert judgment is often accomplished with the use of risk facilitation workshops or interview

In [11]:
def clean(data):
    L = []
    data = data.replace("11 - PROJECT RISK MANAGEMENT\n","") 
    data = data.replace("e.g.,","such as")
    data = data.replace("i.e,","")
    data = data.replace('Work performance data related to various performance results possibly impacted \nby risks includes, but is not limited to:','Work_Performance_Data related to Performance_Results.Work_Performance_Data include:')
    data = data.replace(' Enterprise environmental factors may provide insight and context to the risk analysis, \nsuch as:','Entreprise_Environmental_Factors provide Context_To_Risk such as:')
    data = data.replace(' Enterprise environmental factors may provide insight and context to the risk \nassessment, such as:','Entreprise_Environmental_Factors provide Context_To_Risk such as:')
    data = data.replace('To ensure a comprehensive establishment of the risk management plan, judgment, and expertise should be \nconsidered from groups or individuals with specialized training or knowledge on the subject area, such as:','Expert_Judgment considered from Groups_Or_Individuals such as:')
    data = data.replace("•\t   ","•\t ")
    data = data.replace("•\t  ","•\t ")
    data = data.replace("•\t Interviewing.Interviewing techniques","Data_Gathering_And_Representation_Techniques include:\n•\t Interviewing.Interviewing techniques")
    data = data.replace("'s","")
    data = data.replace('Risks process include, \nbut are not limited to:',"Risks process include:")
    data = data.replace('this process include, \nbut are not limited to:',"this process include:")
    data = data.replace('Change requests can \ninclude recommended corrective and preventive actions as well.','Change_Requests include:')
    data = data.replace('These \nstrategies, described below, are to exploit, share, enhance, and accept.','Strategies_For_Positive_Risks_Or_Opportunities include:')
    data = data.replace("The four strategies for dealing with negative risks or threats are further described as follows","Strategies_For_Negative_Risks_Or_Threats include")
    data = data.replace(', including',' include')
    data = data.replace(', but are not limited to,','')
    data = data.replace(', but are not limited to','')
    data = data.replace('processes:','processes.Risk_Register include:')
    data = data.replace(' become ',' is ')
    data = data.replace(' becomes ',' is ')
    data = data.replace('includes the following','include')
    data = data.replace('(i.e., analysis under uncertainty)','')
    data = data.replace('(i.e., pose a risk to the project)','')
    data = data.replace("Licensed To: Jorge Diego Fuentes Sanchez PMI MemberID: 2399412\nThis copy is a PMI Member benefit, not for distribution, sale, or reproduction.","")
    data = data.replace('11\n','\n')
    data = data.replace(', such as',' such as')
    for i in range(334,380):
        if i in dictionary['the whole page'] or i in dictionary['2 images on the buttom'] or i in dictionary['one image on the buttom']:
            continue
        else:
            if (i-25)%2==0:
                data = data.replace(str(i-25)+" ©2013 Project Management Institute. A Guide to the Project Management Body of Knowledge (PMBOK® Guide) – Fifth Edition \n","")
            else:
                data = data.replace("©2013 Project Management Institute. A Guide to the Project Management Body of Knowledge (PMBOK® Guide) – Fifth Edition  "+str(i-25)+"\n","")
            
    return data
df1 = clean(df1)

In [12]:
df1 = df1.lower()
dicti ={"doesn't":"does not", "can't":"can not", "won't":"will not", "don't":"do not", "i've":"i have", "i'd":"i would", "i'm":"i am", "i'll":"i will", "she's":"she has", "he's":"he has",
        "it's":"it has", "there's":"there is", "they're":"they are", "we're":"we are", "you've":"you have", "you're":"you are", "couldn't":"could not", "shouldn't":"should not",}
for i,j in dicti.items():
    if df1.find(i)!=-1:
        df1 = df1.replace(i,j)
df1 = df1.replace("’s","")
df1 = df1.replace("'s","")

In [13]:
df1 = df1.replace('©2013 project management institute. a guide to the project management body of knowledge (pmbok® guide) – fifth edition  3\n','')

In [14]:
# df1 = df1.replace('©2013 project management institute. a guide to the project management body of knowledge (pmbok® guide) – fifth edition  309\nlicensed to: jorge diego fuentes sanchez pmi memberid: 2399412\nthis copy is a pmi member benefit, not for distribution, sale, or reproduction.','')

In [15]:
df1 = df1.replace('©2013 project management institute. a guide to the project management body of knowledge (pmbok® guide) – fifth edition  3\n','')

In [16]:
df1.find('©2013')

-1

In [17]:
df1.find('•\t  ')

-1

In [18]:
df1[6195:]

'1 plan risk management\nplan risk management is the process of defining how to conduct risk management activities for a project. the \nkey benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate \nwith both the risks and the importance of the project to the organization. the risk management plan is vital to \ncommunicate with and obtain agreement and support from all stakeholders to ensure the risk management process \nis supported and performed effectively over the project life cycle. the inputs, tools and techniques, and outputs of \nthis process are depicted in figure 11-2. figure 11-3 depicts the data flow diagram of the process.\ncareful and explicit planning enhances the probability of success for other risk management processes. planning \nis also important to provide sufficient resources and time for risk management activities and to establish an agreed-\nupon basis for evaluating risks. the plan risk management process s

In [19]:
processes = ['11.1 plan risk management', '11.2 identify risks', '11.3 perform qualitative risk analysis', '11.4 perform quantitative risk analysis', '11.5 plan risk responses', 
     '11.6 control risks']
for p in processes:
    i = 1
    nb = 0
    while True:
        print(p[5:]+':\n')
        i = df1.lower().find(p,i) + 1
        if i==0:
            print('\n number = '+str(nb))
            break
        else:
            print(df1[i-1:i+250])
            nb+=1

plan risk management:

11.1 plan risk management
plan risk management is the process of defining how to conduct risk management activities for a project. the 
key benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate
plan risk management:


 number = 1
identify risks:

11.2 identify risks
identify risks is the process of determining which risks may affect the project and documenting their 
characteristics. the key benefit of this process is the documentation of existing risks and the knowledge and ability 
it provid
identify risks:


 number = 1
perform qualitative risk analysis:

11.3 perform qualitative risk analysis
perform qualitative risk analysis is the process of prioritizing risks for further analysis or action by assessing 
and combining their probability of occurrence and impact. the key benefit of this process is tha
perform qualitative risk analysis:


 number = 1
perform quantitative risk analysis:

11.4 perform qua

In [20]:
list_of_processes = []
df2 = df1[:]
for i in np.arange(-1,-7,-1):
    ch = ''
    indice = df2.lower().find(processes[i])
    ch = df2[indice+len(processes[i]+'\n'):]
    df2 = df2[:indice]
    list_of_processes.append(ch)
list_of_processes

['control risks is the process of implementing risk response plans, tracking identified risks, monitoring residual \nrisks, identifying new risks, and evaluating risk process effectiveness throughout the project. the key benefit of \nthis process is that it improves efficiency of the risk approach throughout the project life cycle to continuously \noptimize risk responses. the inputs, tools and techniques, and outputs of this process are depicted in figure 11-20. \nfigure 11-21 depicts the data flow diagram of the process.\nplanned risk responses that are included in the risk register are executed during the life cycle of the project, but \nthe project work should be continuously monitored for new, changing, and outdated risks.\nthe control risks process applies techniques such as variance and trend analysis, which require the use of \nperformance information generated during project execution. other purposes of the control risks process are to \ndetermine if:\n•\t project assumptions 

In [21]:
list_of_processes[5][-500:]

', as they apply to the specific project, may \nbe revised in the plan risk management process.\n•\t reporting formats. reporting formats define how the outcomes of the risk management process will \nbe documented, analyzed, and communicated. it describes the content and format of the risk register as \nwell as any other risk reports required.\n•\t tracking. tracking documents how risk activities will be recorded for the benefit of the current project \nand how risk management processes will be audited.\n'

In [22]:
list_of_processes.reverse()

In [23]:
dictionary_of_processes = dict()
L = range(len(processes))
for k in L:
    dictionary_of_processes.update({processes[k][5:]:list_of_processes[k]})

In [24]:
dictionary_of_processes

{'plan risk management': 'plan risk management is the process of defining how to conduct risk management activities for a project. the \nkey benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate \nwith both the risks and the importance of the project to the organization. the risk management plan is vital to \ncommunicate with and obtain agreement and support from all stakeholders to ensure the risk management process \nis supported and performed effectively over the project life cycle. the inputs, tools and techniques, and outputs of \nthis process are depicted in figure 11-2. figure 11-3 depicts the data flow diagram of the process.\ncareful and explicit planning enhances the probability of success for other risk management processes. planning \nis also important to provide sufficient resources and time for risk management activities and to establish an agreed-\nupon basis for evaluating risks. the plan risk management process 

In [25]:
df1[27:]

'project risk management includes the processes of conducting risk management planning, identification, \nanalysis, response planning, and controlling risk on a project. the objectives of project risk management are to \nincrease the likelihood and impact of positive events, and decrease the likelihood and impact of negative events \nin the project.\nfigure 11-1 provides an overview of the project risk management processes, which are as follows:\n11.1  plan risk management—the process of defining how to conduct risk management activities for a \nproject.\n11.2  identify risks—the process of determining which risks may affect the project and documenting \n\ntheir characteristics.\n11.3  perform qualitative risk analysis—the process of prioritizing risks for further analysis or action \nby assessing and combining their probability of occurrence and impact.\n11.4  perform quantitative risk analysis—the process of numerically analyzing the effect of identified \nrisks on overall project ob

In [26]:
df1 = df1[27:]

In [27]:
df1[:6164]

'project risk management includes the processes of conducting risk management planning, identification, \nanalysis, response planning, and controlling risk on a project. the objectives of project risk management are to \nincrease the likelihood and impact of positive events, and decrease the likelihood and impact of negative events \nin the project.\nfigure 11-1 provides an overview of the project risk management processes, which are as follows:\n11.1  plan risk management—the process of defining how to conduct risk management activities for a \nproject.\n11.2  identify risks—the process of determining which risks may affect the project and documenting \n\ntheir characteristics.\n11.3  perform qualitative risk analysis—the process of prioritizing risks for further analysis or action \nby assessing and combining their probability of occurrence and impact.\n11.4  perform quantitative risk analysis—the process of numerically analyzing the effect of identified \nrisks on overall project ob

In [28]:
overview = df1[:6164]

In [29]:
overview[-500:]

'.\nto be successful, an organization should be committed to address risk management proactively and consistently \nthroughout the project. a conscious choice should be made at all levels of the organization to actively identify and \npursue effective risk management during the life of the project. project risk could exist at the moment a project \nis initiated. moving forward on a project without a proactive focus on risk management is likely to lead to more \nproblems arising from unmanaged threats.'

In [30]:
global_dict = {'name':'overview','text':overview,'Concept':'PlanRiskManagement'}

In [31]:
import spacy
sp = spacy.load('en_core_web_sm')

In [32]:
amod_to_keep = ['Quantitative','identified','environmental', 'external', 'Individual', 'individual', 'organizational','unmanaged', 'Analytical', 'analytical', 'Subject', 'Senior', 'senior', 'subject', 'Professional', 'professional', 'quantitative', 'moderate', 'Organizational', 'delphi', 'preventive', 'historical', 'diagramming', 'graphical', 'Potential', 'potential', 'Expected', 'expected',  'Prioritized', 'prioritized', 'Probabilistic', 'probabilistic', 'monetary', 'quantified', 'Human', 'human', 'Deliverable', 'deliverable', 'Actual', 'actual']

In [33]:
def pre_compound(text):
    dictn = {}
    L = text.split(' ')
    for j in range(len(L)):
        if '-' not in L[j]:
            continue
        elif '-' in L[j] and not any(c.isdigit() for c in L[j]):
            l = L[j].split('-')
            try:
                l2 = [sp(word)[0].pos_ for word in l]
            except:
                continue
            if 'NOUN' in l2 or 'VERB' in l2:
                bol = True
            else:
                bol = False
            dictn[j] = bol
    for k,v in dictn.items():
        if v == True:
            L[k] = L[k].replace('-','_')
            gg = [w[0].upper()+w[1:] for w in L[k].split('_')]
            L[k] = '_'.join(gg)
        else:
            L[k] = L[k].replace('-',' ')
    return ' '.join(L)

In [34]:
def compound_noun(ch):
    sp = spacy.load('en_core_web_sm')
    doc = sp(ch)
    text = ''
    ind = 0
    for word in range(len(doc)):
        if word<ind :
            continue

        if ((doc[word].dep_!='compound' or (doc[word].dep_=='compound' and doc[word].pos_=='SPACE'))  and doc[word].pos_!='NOUN' and doc[word].pos_!='PROPN' and (doc[word].dep_!='amod' or (doc[word].dep_=='amod' and doc[word].text not in amod_to_keep))and (doc[word].dep_!='nmod' or (doc[word].dep_=='nmod' and doc[word].pos_=='SPACE'))) or doc[word].text=='•' or doc[word].text=='\n':
            if word+1<len(doc)-1:
                if doc[word+1].pos_ in ['PUNCT','SPACE'] or doc[word+1].text=='•' or doc[word+1].text=='\n':
                    text+=str(doc[word].text)
                else:
                    text+=str(doc[word].text) + ' '
            else:
                if word==len(doc)-1:
                    text+=str(doc[word].text)
                else:
                    if doc[word+1].pos_ in ['PUNCT','SPACE'] or doc[word+1].text=='•' or doc[word+1].text=='\n':
                        text+=str(doc[word].text)+str(doc[word+1].text)
                    else:
                        text+=str(doc[word].text) + ' ' + str(doc[word+1].text)
                break
        elif ind == len(doc)-1 and doc[ind].text in text.lower().split()[-1]:
            break
        else:
            compound = str(doc[word].text)
            rest_of_comp=[]
            for sub_tok in range(word+1,len(doc)):
                punc=''
                if doc[sub_tok].pos_=='NOUN' or doc[sub_tok].pos_=='PROPN' or doc[sub_tok].dep_=='compound' or doc[sub_tok].dep_=='nmod' or (doc[sub_tok].dep_=='amod' and  doc[sub_tok].text in amod_to_keep):
                    rest_of_comp.append(str(doc[sub_tok].text)[0].upper()+str(doc[sub_tok].text)[1:])
                    ind = sub_tok
                elif doc[sub_tok].pos_ in ['SPACE','PUNCT'] or doc[word+1].text=='•' or doc[word+1].text=='\n':
                    if sub_tok == len(doc)-1:
                        ind = sub_tok
                    else:
                        if doc[sub_tok].text=='\n':
                            continue
                        elif doc[sub_tok].pos_!='SPACE':
                            punc = str(doc[sub_tok].text)    
                        ind = sub_tok+1
                        break
                else:
                    ind = sub_tok
                    break
            if doc[ind].pos not in ['PUNCT','SPACE'] or doc[word+1].text=='•':
                if rest_of_comp:
                    text+= compound[0].upper()+compound[1:]+'_'+'_'.join(rest_of_comp)+punc+' '
                else:
                    text+=compound+punc+' '
            else:
                print('punc:',doc[ind])
                if rest_of_comp:
                    text+= compound[0].upper()+compound[1:]+'_'+'_'.join(rest_of_comp)+punc
                else:
                    text+=compound+punc
    text = text.replace('. \n','.\n')
    return text

In [35]:
compound_noun('project risk management')

'Project_Risk_Management '

In [36]:
global_dict['text'] = pre_compound(global_dict['text'].lower())

In [37]:
global_dict['text'] = compound_noun(global_dict['text'].lower())

In [38]:
global_dict['text'] = global_dict['text'].replace('_’s','')

In [39]:
global_dict['text']

'Project_Risk_Management includes the processes of conducting Risk_Management_Planning, identification, \n analysis, Response_Planning, and controlling risk on a project. the objectives of Project_Risk_Management are to\n increase the likelihood and impact of positive events, and decrease the likelihood and impact of negative eventsin the project.\n figure 11 - 1 provides an overview of the Project_Risk_Management_Processes, which are as follows:\n 11.1  Plan_Risk_Management— the process of defining how to conduct Risk_Management_Activities for a\n project.\n 11.2  identify risks— the process of determining which risks may affect the project and documenting\n\n their characteristics.\n 11.3  perform qualitative Risk_Analysis— the process of prioritizing risks for further analysis or actionby assessing and combining their probability of occurrence and impact.\n 11.4  perform Quantitative_Risk_Analysis— the process of numerically analyzing the effect of Identified_\n_Riskson overall Proj

In [40]:
compound_noun('Described in Section 4.1.3.1. The project charter.')

'Described in Section 4.1.3.1. The Project_Charter .'

In [513]:
def relevant_concepts(process,name):
    final = []
    i = -1
    process = '\n'+process
    reg = re.compile(r'\n\d+\.(\d+)\.(\d+)\s[A-Za-z]+')
    matches = reg.finditer(process)
    for match in matches:
        print(match.group())
        spans = match.span()
        group = match.group()
        type1 = process[spans[0]+1:process.find('\n',spans[0]+1)].split(':')[1][1:].lower()
        if match.group(2)=='1':
            process_desc = process[1:spans[0]]
        reg2 = re.compile(r'\n\d+\.(\d+)\.(\d+)\.(\d+)\s[A-Za-z]+')
        matchess = reg2.finditer(process)
        for other_match in matchess:
            if other_match.group(2)!=match.group(2):
                continue
            else:
                print(other_match.group())
                i = i+1
                new = []
                new_dict = {'concept':name, 'type':type1, 'process_desc':process_desc}
                spans2 = other_match.span()
                group = other_match.group()
                new_name = process[spans2[0]+1:process.find('\n',spans2[0]+1)].split(' ',1)[1].lower()
                find = '\n11.'+other_match.group(1)+'.'+other_match.group(2)+'.'+str(int(other_match.group(3))+1)
                if process.find(find)==-1:
                    find = '\n11.'+other_match.group(1)+'.'+str(int(other_match.group(2))+1)
                end = process.find(find)
                if end!=-1:
                    value = process[process.find('\n',spans2[0]+1)+1:end+1]
                else:
                    value = process[process.find('\n',spans2[0]+1)+1:]
                new_dict['subconcept'] = new_name
                new_dict['content'] = value
                for k,v in new_dict.items():
                    new.append(str(v))
                final.append(new)
    dataaa = pd.DataFrame(final, columns=['concept','type', 'process_desc', 'subconcept', 'content'],dtype=str)
    return dataaa

In [514]:
data = pd.DataFrame(columns=['concept','type', 'process_desc', 'subconcept', 'content'])
for i in dictionary_of_processes.keys():
    process = dictionary_of_processes[i]
    dataa = relevant_concepts(process,i)
    data = pd.concat([data,dataa], ignore_index=True)


11.1.1 plan

11.1.1.1 project

11.1.1.2 project

11.1.1.3 stakeholder

11.1.1.4 enterprise

11.1.1.5 organizational

11.1.2 plan

11.1.2.1 analytical

11.1.2.2 expert

11.1.2.3 meetings

11.1.3 plan

11.1.3.1 risk

11.2.1 identify

11.2.1.1 risk

11.2.1.2 cost

11.2.1.3 schedule

11.2.1.4 quality

11.2.1.5 human

11.2.1.6 scope

11.2.1.7 activity

11.2.1.8 activity

11.2.1.9 stakeholder

11.2.1.10 project

11.2.1.11 procurement

11.2.1.12 enterprise

11.2.1.13 organizational

11.2.2 identify

11.2.2.1 documentation

11.2.2.2 information

11.2.2.3 checklist

11.2.2.4 assumptions

11.2.2.5 diagramming

11.2.2.6 swot

11.2.2.7 expert

11.2.3 identify

11.2.3.1 risk

11.3.1 perform

11.3.1.1 risk

11.3.1.2 scope

11.3.1.3 risk

11.3.1.4 enterprise

11.3.1.5 organizational

11.3.2 perform

11.3.2.1 risk

11.3.2.2 probability

11.3.2.3 risk

11.3.2.4 risk

11.3.2.5 risk

11.3.2.6 expert

11.3.3 perform

11.3.3.1 project

11.4.1 perform

11.4.1.1 risk

11.4.1.2 cost

11.4.1.3 schedule

11.4.

In [515]:
data= data.convert_dtypes()

In [516]:
data.dtypes

concept         string
type            string
process_desc    string
subconcept      string
content         string
dtype: object

In [517]:
def removeUselessSlashN(ch):
    new_ch=ch[0]
    for c in range(1,len(ch)):
        if (ch[c]=="\n" and ch[c-1]!='.') or ch[c]=="\t":
            continue
        new_ch+=ch[c]
    return new_ch

In [518]:
data['content'] = data['content'].apply(lambda ch:removeUselessSlashN(ch))

In [519]:
data['content'][4]

'described in section 2.1.4. the organizational process assets that can influence the plan risk management process include:• risk categories,• common definitions of concepts and terms,• risk statement formats,• standard templates,• roles and responsibilities,• authority levels for decision making, and• lessons learned.\n'

In [520]:
data['concept'][0]

'plan risk management'

In [521]:
l = [{} for i in data.index]
data['infos'] = l

In [522]:
def extract_section(data_input,i):
    reg = re.compile(r'((described)|(defined)) in section (\d+\.){3,4} ')
    if len(list(reg.finditer(data_input.at[i,'content'].lower())))==0:
        pass
    else:
        matches = reg.finditer(data_input.at[i,'content'].lower())
        all_matches = {}
        b=-1
        for match in matches:
            b+=1
            spans = match.span()
            group = match.group()
            describe = group
            who = str(data_input.at[i,'concept'])+' '+str(data_input.at[i,'subconcept'])
            my_dict = {'about':who,'description':describe}
            all_matches[b] = my_dict
            data['content'][i] = data['content'][i][spans[1]:]
        if data_input['infos'][i]=={}:
            data_input['infos'][i] = all_matches
        else:
            k = list(data_input['infos'][i].keys())[-1]
            existants = []
            for k,v in data_input['infos'][i].items():
                existants.append(str(v['description']))
            for k1 in all_matches.keys():
                if all_matches[k1]['description'] not in existants:
                    k = k+1
                    data_input.loc[i,'infos'][k] = all_matches[k1]
    return data_input
for i in data.index:
    data = extract_section(data,i)
data.head()

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,concept,type,process_desc,subconcept,content,infos
0,plan risk management,inputs,plan risk management is the process of definin...,project management plan,"in planning risk management, all approved subs...",{}
1,plan risk management,inputs,plan risk management is the process of definin...,project charter,the project charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,plan risk management,inputs,plan risk management is the process of definin...,stakeholder register,"the stakeholder register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,plan risk management,inputs,plan risk management is the process of definin...,enterprise environmental factors,the enterprise environmental factors that can ...,{0: {'about': 'plan risk management enterprise...
4,plan risk management,inputs,plan risk management is the process of definin...,organizational process assets,the organizational process assets that can inf...,{0: {'about': 'plan risk management organizati...


In [523]:
data['infos'][1]

{0: {'about': 'plan risk management project charter',
  'description': 'described in section 4.1.3.1. '}}

In [524]:
data.shape

(75, 6)

In [525]:
def find_table(input_data):
    sentences = []
    for i in input_data.index:
        if input_data['content'][i].lower().find('table 11')==-1:
            continue
        else:
            circle_pos = 0
            dot_pos = 0
            circles = []
            sentences = []
            dicto = {}
            while circle_pos!=-1:
                circle_pos = input_data['content'][i].find('•',circle_pos+1)
                if circle_pos!=-1:
                    circles.append(circle_pos)
            sentences = [input_data['content'][i][circles[j]+2:circles[j+1]-1] for j in range(len(circles)-1)]
            for text in sentences:
                if text.find('table 11')!=-1:
                    title,defi = text.split('.',1)
                    def_list = defi.split('.')
                    defi = '.'.join([ch for ch in def_list if ch.find('table 11')!=-1])
            dicto.update({'about':title,'description':defi})
            existants = []
            for k,v in input_data['infos'][i].items():
                existants.append(str(v['description']))
            k=-1
            if input_data['infos'][i]!={}:
                k = list(input_data['infos'][i].keys())[-1]
            if dicto['description'] not in existants:
                input_data['infos'][i][k+1] = dicto
    return input_data

In [526]:
data = find_table(data)

In [527]:
def remove_table(input_data):
    for i in input_data.index:
        if input_data['content'][i].lower().find('table 11')==-1:
            continue
        else:
            L = input_data['content'][i].split('.')
            new = '.'.join([sentence for sentence in L if sentence.lower().find('table 11')==-1])
            input_data['content'][i] = new
    return input_data
data = remove_table(data)
data.head()

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,concept,type,process_desc,subconcept,content,infos
0,plan risk management,inputs,plan risk management is the process of definin...,project management plan,"in planning risk management, all approved subs...",{}
1,plan risk management,inputs,plan risk management is the process of definin...,project charter,the project charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,plan risk management,inputs,plan risk management is the process of definin...,stakeholder register,"the stakeholder register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,plan risk management,inputs,plan risk management is the process of definin...,enterprise environmental factors,the enterprise environmental factors that can ...,{0: {'about': 'plan risk management enterprise...
4,plan risk management,inputs,plan risk management is the process of definin...,organizational process assets,the organizational process assets that can inf...,{0: {'about': 'plan risk management organizati...


In [528]:
annotations = {'about':[],'description':[]}
for i in data.index:
    if data['infos'][i]!={}:
        dicto = data['infos'][i]
        for k,v in dicto.items():
            annotations['about'].append(v['about'])
            annotations['description'].append(v['description'])
annotations = pd.DataFrame(annotations)
annotations['about'] = annotations['about'].apply(lambda ch: '_'.join([word[0].upper()+word[1:]for word in ch.split(' ') if word!='']))
annotations

,about,description
0,Plan_Risk_Management_Project_Charter,described in section 4.1.3.1.
1,Plan_Risk_Management_Stakeholder_Register,described in section 13.1.3.1.
2,Plan_Risk_Management_Enterprise_Environmental_...,described in section 2.1.5.
3,Plan_Risk_Management_Organizational_Process_As...,described in section 2.1.4.
4,Definitions_Of_Risk_Probability_And_Impact,table 11-1 is an example of definitions of ne...
5,Identify_Risks_Risk_Management_Plan,described in section 11.1.3.1.
6,Identify_Risks_Cost_Management_Plan,described in section 7.1.3.1.
7,Identify_Risks_Schedule_Management_Plan,described in section 6.1.3.1.
8,Identify_Risks_Quality_Management_Plan,described in section 8.1.3.1.
9,Identify_Risks_Human_Resource_Management_Plan,described in section 9.1.3.1.


In [529]:
compound_noun(text)

'final versions of the Risk_Register and the Risk_Management_Plan_Templates, checklists, and Risk_Breakdown_Structure are included'

In [530]:
data['infos'][9]

{0: {'about': 'identify risks risk management plan ',
  'description': 'described in section 11.1.3.1. '}}

In [531]:
data['infos'][49]

{}

In [532]:
indexes = []
for i in data.index:
    if data['content'][i] in [' ','.','\n']:
        indexes.append(i)
data.drop(index=indexes,inplace=True)
data = data.reset_index(drop=True)

In [533]:
data.head(10)

,concept,type,process_desc,subconcept,content,infos
0,plan risk management,inputs,plan risk management is the process of definin...,project management plan,"in planning risk management, all approved subs...",{}
1,plan risk management,inputs,plan risk management is the process of definin...,project charter,the project charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,plan risk management,inputs,plan risk management is the process of definin...,stakeholder register,"the stakeholder register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,plan risk management,inputs,plan risk management is the process of definin...,enterprise environmental factors,the enterprise environmental factors that can ...,{0: {'about': 'plan risk management enterprise...
4,plan risk management,inputs,plan risk management is the process of definin...,organizational process assets,the organizational process assets that can inf...,{0: {'about': 'plan risk management organizati...
5,plan risk management,tools and techniques,plan risk management is the process of definin...,analytical techniques,analytical techniques are used to understand a...,{}
6,plan risk management,tools and techniques,plan risk management is the process of definin...,expert judgment,expert_judgment considered from groups_or_indi...,{}
7,plan risk management,tools and techniques,plan risk management is the process of definin...,meetings,project teams hold planning meetings to develo...,{}
8,plan risk management,outputs,plan risk management is the process of definin...,risk management plan,the risk management plan is a component of the...,{0: {'about': 'definitions of risk probability...
9,identify risks,inputs,identify risks is the process of determining w...,risk management plan,key elements of the risk management plan that ...,{0: {'about': 'identify risks risk management ...


In [534]:
for i in data.index:
    l = data['subconcept'][i].strip().split()
    l = [ch[0].upper()+ch[1:] for ch in l if ch not in ['', ' ']]
    data['subconcept'][i] = '_'.join(l)

In [535]:
indexes=[]
for i in data.index:
    if data['content'][i].find('such as:')!=-1:
        indexes.append(i)
for x in indexes:
    print(x,'\n',data['content'][x],'\n\n')

6 
 expert_judgment considered from groups_or_individuals such as:• senior management,• project stakeholders,• project managers who have worked on projects in the same area (directly or through lessons learned),• subject matter experts (smes) in business or project area,• industry groups and consultants, and• professional and technical associations.
 


33 
 described in section 2.1.5.entreprise_environmental_factors provide context_to_risk such as:• industry studies of similar projects by risk specialists, and• risk databases that may be available from industry or proprietary sources.
 


46 
 described in section 2.1.5.entreprise_environmental_factors provide context_to_risk such as:• industry studies of similar projects by risk specialists, and• risk databases that may be available from industry or proprietary sources.
 




In [536]:
for i in data.index:
    text = data['content'][i]
    ref = ' '.join(data['subconcept'][i].split('_')).lower()
    text = text.lower().replace(ref,data['subconcept'][i])
    data['content'][i] = text

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [537]:
data['content'][4]

'the Organizational_Process_Assets that can influence the plan risk management process include:• risk categories,• common definitions of concepts and terms,• risk statement formats,• standard templates,• roles and responsibilities,• authority levels for decision making, and• lessons learned.\n'

In [538]:
data['content'] = data['content'].apply(lambda ch:pre_compound(ch))
data['content'] = data['content'].apply(lambda ch:compound_noun(ch))

In [539]:
data['content'][3]

'the Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.\n'

In [540]:
data['content'] = data['content'].apply(lambda ch:ch.replace('\t  ','\t '))

In [541]:
data['content'] = data['content'].apply(lambda ch:ch.replace('_Include',' include'))
data['content'] = data['content'].apply(lambda ch:ch.replace('include, but are not limitedto:',' include:'))
data['content'] = data['content'].apply(lambda ch:ch.replace('risk owners','Risk_Owners'))

In [542]:
data['concept'] = data['concept'].apply(lambda ch:pre_compound(ch))
data['concept'] = data['concept'].apply(lambda ch:compound_noun(ch))

In [543]:
data

,concept,type,process_desc,subconcept,content,infos
0,plan Risk_Management,inputs,plan risk management is the process of definin...,Project_Management_Plan,"in planning Risk_Management, all approved Subs...",{}
1,plan Risk_Management,inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,plan Risk_Management,inputs,plan risk management is the process of definin...,Stakeholder_Register,"the Stakeholder_Register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,plan Risk_Management,inputs,plan risk management is the process of definin...,Enterprise_Environmental_Factors,the Enterprise_Environmental_Factors that can ...,{0: {'about': 'plan risk management enterprise...
4,plan Risk_Management,inputs,plan risk management is the process of definin...,Organizational_Process_Assets,the Organizational_Process_Assets that can inf...,{0: {'about': 'plan risk management organizati...
...,...,...,...,...,...,...
70,Control_Risks,outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information, as a Control_Ris...",{}
71,Control_Risks,outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds ...,{}
72,Control_Risks,outputs,control risks is the process of implementing r...,Project_Management_Plan_Updates,if the approved Change_Requests have an effect...,{}
73,Control_Risks,outputs,control risks is the process of implementing r...,Project_Documents_Updates,Project_Documents that may be updated as a res...,{}


In [544]:
def format(data,columns):
    for c in columns:
        for r in data.index:
            l = []
            data[c][r] = data[c][r].strip()
            if ' ' in data[c][r]:
                l1 = data[c][r].split(' ')
                for x in l1:
                    x = x.split('_')
                    l.extend(x)
                l = [ch[0].upper()+ch[1:] for ch in l]
                text = '_'.join(l)
                data[c][r] = text
            else:
                l = data[c][r].strip().split('_')
                l = [ch[0].upper()+ch[1:] for ch in l]
                text = '_'.join(l)
                data[c][r] = text
    return data
data = format(data,['concept','type','subconcept'])

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [545]:
def format_subconcept(data):
    sub = list(data['subconcept'])
    unique = [x for x in sub if sub.count(x)==1]
    dict_to_iterate = {k:v for k,v in enumerate(sub)}
    for k,v in dict_to_iterate.items():
        if v in unique:
            continue
        data['subconcept'][k] = data['concept'][k]+'_'+data['subconcept'][k]
    return data
data = format_subconcept(data)

In [546]:
data

,concept,type,process_desc,subconcept,content,infos
0,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management, all approved Subs...",{}
1,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,"the Stakeholder_Register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Enterprise_Environmental_Factors that can ...,{0: {'about': 'plan risk management enterprise...
4,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,the Organizational_Process_Assets that can inf...,{0: {'about': 'plan risk management organizati...
...,...,...,...,...,...,...
70,Control_Risks,Outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information, as a Control_Ris...",{}
71,Control_Risks,Outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds ...,{}
72,Control_Risks,Outputs,control risks is the process of implementing r...,Control_Risks_Project_Management_Plan_Updates,if the approved Change_Requests have an effect...,{}
73,Control_Risks,Outputs,control risks is the process of implementing r...,Control_Risks_Project_Documents_Updates,Project_Documents that may be updated as a res...,{}


In [547]:
annotations

,about,description
0,Plan_Risk_Management_Project_Charter,described in section 4.1.3.1.
1,Plan_Risk_Management_Stakeholder_Register,described in section 13.1.3.1.
2,Plan_Risk_Management_Enterprise_Environmental_...,described in section 2.1.5.
3,Plan_Risk_Management_Organizational_Process_As...,described in section 2.1.4.
4,Definitions_Of_Risk_Probability_And_Impact,table 11-1 is an example of definitions of ne...
5,Identify_Risks_Risk_Management_Plan,described in section 11.1.3.1.
6,Identify_Risks_Cost_Management_Plan,described in section 7.1.3.1.
7,Identify_Risks_Schedule_Management_Plan,described in section 6.1.3.1.
8,Identify_Risks_Quality_Management_Plan,described in section 8.1.3.1.
9,Identify_Risks_Human_Resource_Management_Plan,described in section 9.1.3.1.


In [548]:
def format_about(data,annotations):
    sub = list(data['subconcept'])
    to = {}
    actual = []
    for i in data.index:
        if data['subconcept'][i].find(data['concept'][i])==-1:
            to[i] = data['subconcept'][i]
            actual.append(data['concept'][i]+'_'+data['subconcept'][i])
    actual2 = {}
    for x in annotations.index:
        if annotations['description'][x].find('table 11-1')!=-1:
            continue
        if annotations['about'][x] in actual:
            actual2[x] = annotations['about'][x]
    for k1,v1 in actual2.items():
        for k2,v2 in to.items():
            if v1.find(v2)!=-1:
                annotations['about'][k1] = data['subconcept'][k2]
    return annotations
annotations = format_about(data,annotations)
annotations

,about,description
0,Project_Charter,described in section 4.1.3.1.
1,Plan_Risk_Management_Stakeholder_Register,described in section 13.1.3.1.
2,Plan_Risk_Management_Enterprise_Environmental_...,described in section 2.1.5.
3,Plan_Risk_Management_Organizational_Process_As...,described in section 2.1.4.
4,Definitions_Of_Risk_Probability_And_Impact,table 11-1 is an example of definitions of ne...
5,Identify_Risks_Risk_Management_Plan,described in section 11.1.3.1.
6,Identify_Risks_Cost_Management_Plan,described in section 7.1.3.1.
7,Identify_Risks_Schedule_Management_Plan,described in section 6.1.3.1.
8,Quality_Management_Plan,described in section 8.1.3.1.
9,Human_Resource_Management_Plan,described in section 9.1.3.1.


In [549]:
about = ['Expected_Monetary_Value_Analysis', 'Influence_Diagrams', 'Interview', 'Probability_And_Impact_Matrix', 'Probability_Of_Achieving_Cost_And_Time_Objectives', 'Risk', 'Risk_Breakdown_Structure']
desc = ['A common use of  this type of analysis is a decision tree analysis (Figure 11-16)', 'as shown in Figure 11-7.', 'Examples of three point estimates for cost are shown in Figure 11-13.', 'in Figure 11-10,', 'For instance, in Figure 11-17, the likelihood of achieving the cost estimate of US$41 million is about 12%', 'risk is an uncertain event or condition that, if it occurs, has a positive or negative effect on one or more project objectives such as scope, schedule, cost, and quality.', 'An example is shown in Figure 11-4.']
anno = {'about':about,'description':desc}
anno = pd.DataFrame(anno)
annotations = pd.concat([annotations,anno],ignore_index=True)
desc = data[['concept','process_desc']].drop_duplicates()
desc.rename(columns={'concept':'about','process_desc':'description'},inplace=True)
annotations = pd.concat([annotations,desc],ignore_index=True)
annotations.shape

(42, 2)

In [550]:
'Plan_Risk_Management_Project_Charter'.split('Plan_Risk_Management')

['', '_Project_Charter']

In [551]:
data

,concept,type,process_desc,subconcept,content,infos
0,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management, all approved Subs...",{}
1,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various inputs...,{0: {'about': 'plan risk management project ch...
2,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,"the Stakeholder_Register, which contains all d...",{0: {'about': 'plan risk management stakeholde...
3,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Enterprise_Environmental_Factors that can ...,{0: {'about': 'plan risk management enterprise...
4,Plan_Risk_Management,Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,the Organizational_Process_Assets that can inf...,{0: {'about': 'plan risk management organizati...
...,...,...,...,...,...,...
70,Control_Risks,Outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information, as a Control_Ris...",{}
71,Control_Risks,Outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds ...,{}
72,Control_Risks,Outputs,control risks is the process of implementing r...,Control_Risks_Project_Management_Plan_Updates,if the approved Change_Requests have an effect...,{}
73,Control_Risks,Outputs,control risks is the process of implementing r...,Control_Risks_Project_Documents_Updates,Project_Documents that may be updated as a res...,{}


In [552]:
def format_content(data):
    values = [tuple([data['concept'][k],data['subconcept'][k]]) for k in data.index]
    old_values = []
    for i in range(len(values)):
        elem = values[i][1].split(values[i][0])
        if len(elem)==1:
            old_values.append(elem[0])
        else:
            old_values.append(elem[1][1:])
    for i in data.index:
        data['content'][i] = data['content'][i].replace(old_values[i],values[i][1])
    return data
data = format_content(data)

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [553]:
for i in data.index:
    text = data['content'][i]
    l = text.split(' ')
    for x in l:
        if len(x)>0:
            if x[0].islower() or (x[0].isupper() and '_' in x):
                continue
            elif x[0].isupper():
                print(x)
            else:
                print(x)

Needed.

11
-
4.

0.10.0
0.0beta
.
11
-
4.
(
.
•
”
’
’
11
-
4).

/


11
-
7.

’
’
11
-
10,
11
-
10,
.
11
-
10
,
,
11
-
10.
’
•
11
-
13.
6.5)
7.2).
11
-
14.
11
-
15),
11
-
16).
•
11
-
17.
11
-
17,
12
%.
•
,
4.5).
.


In [554]:
for i in data.index:
    text = data['content'][i]
    text = text.replace(' input ',' Inputs ')
    text = text.replace(' inputs ',' Inputs ')
    text = text.replace(' outputs ',' Outputs ')
    text = text.replace(' output ',' Outputs ')
    text = text.replace(' tools and techniques ',' Tools_And_Techniques ')
    text = text.replace(' tool ',' Tools_And_Techniques ')
    text = text.replace(' tools ',' Tools_And_Techniques ')
    text = text.replace(' technique ',' Tools_And_Techniques ')
    data['content'][i] = text

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [555]:
l=[]
for i in data.index:
    text = data['content'][i]
    start=0
    while True:
        start = text.find(' tools and techniques ',start+1)
        if start==-1:
            break
        else:
            l.append('inp')
print(len(l))

0


In [556]:
for i in data.index:
    data['type'][i] = data['concept'][i] + '_' + data['type'][i]
data

,concept,type,process_desc,subconcept,content,infos
0,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management, all approved Subs...",{}
1,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various Inputs...,{0: {'about': 'plan risk management project ch...
2,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,"the Plan_Risk_Management_Stakeholder_Register,...",{0: {'about': 'plan risk management stakeholde...
3,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Plan_Risk_Management_Enterprise_Environmen...,{0: {'about': 'plan risk management enterprise...
4,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,the Plan_Risk_Management_Organizational_Proces...,{0: {'about': 'plan risk management organizati...
...,...,...,...,...,...,...
70,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information, as a Control_Ris...",{}
71,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds ...,{}
72,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Management_Plan_Updates,if the approved Change_Requests have an effect...,{}
73,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Documents_Updates,Project_Documents that may be updated as a res...,{}


In [557]:
data['content'][58]

'elements of the Project_Management_Plan that may be updated as a result of carrying out this process include:• Schedule_Management_Plan. the Schedule_Management_Plan is updated to reflect changes in process and practice driven by the Risk_Responses. this may include changes in tolerance or behavior related to Resource_Loading and leveling, as well as updates to the Schedule_Strategy.\n• Cost_Management_Plan. the Cost_Management_Plan is updated to reflect changes in process and practice driven by the Risk_Responses. this may include changes in tolerance or behavior related to Cost_Accounting, tracking, and reports, as well as updates to the Budget_Strategy and how Contingency_Reserves are consumed.\n• Quality_Management_Plan. the Quality_Management_Plan is updated to reflect changes in process and practice driven by the Risk_Responses. this may include changes in tolerance or behavior related to requirements, Quality_Assurance, or Quality_Control, as well as updates to the Requirements

In [558]:
values = [tuple([data['concept'][k],data['type'][k]]) for k in data.index]
old_values = []
for i in range(len(values)):
    elem = values[i][1].split(values[i][0])
    if len(elem)==1:
        old_values.append(elem[0])
    else:
        old_values.append(elem[1][1:])
for i in data.index:
    data['content'][i] = data['content'][i].replace(old_values[i],values[i][1])

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [559]:
data

,concept,type,process_desc,subconcept,content,infos
0,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management, all approved Subs...",{}
1,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various Plan_R...,{0: {'about': 'plan risk management project ch...
2,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,"the Plan_Risk_Management_Stakeholder_Register,...",{0: {'about': 'plan risk management stakeholde...
3,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Plan_Risk_Management_Enterprise_Environmen...,{0: {'about': 'plan risk management enterprise...
4,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,the Plan_Risk_Management_Organizational_Proces...,{0: {'about': 'plan risk management organizati...
...,...,...,...,...,...,...
70,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information, as a Control_Ris...",{}
71,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds ...,{}
72,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Management_Plan_Updates,if the approved Change_Requests have an effect...,{}
73,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Documents_Updates,Project_Documents that may be updated as a res...,{}


In [90]:
# def suchAsCircle(ch):
#     final_ch=[]
#     pos_such=0
#     pos_such=ch.find(' such as: ',pos_such+1)
#     if pos_such !=-1:
#         pos_commas=[]
#         pos_comma=ch.find("•")
#         while pos_comma!=-1 and pos_comma>pos_such:
#             pos_commas.append(pos_comma)
#             pos_comma=ch.find("•",pos_comma+1)
#         for i in range(len(pos_commas)-1):   
#             final_ch.append(ch[:pos_such+8]+" "+ch[pos_commas[i]+2:pos_commas[i+1]-1])    
#         final_ch.append(ch[:pos_such+8]+" "+ch[pos_commas[-1]+2:ch.find(".",pos_commas[-1])])
#         return ". ".join(final_ch)
#     return ch 

In [91]:
# c = pd.DataFrame(columns=('named entity', 'output'))
# for i in data.index:
#     doc = sp(data['content'][i])
#     d = []
#     for ent in doc.ents:
#         d.append((ent.label_, ent.text))
#     df_entity = pd.DataFrame(d, columns=('named entity', 'output'))
#     c = pd.concat([c,df_entity],ignore_index=True)
# for i in c.index:
#     print(c['named entity'][i],'===========',c['output'][i])

In [560]:
def suchAsCircle(ch):
    final=[]
    pos_include1=ch.lower().find(' such as:')
    if pos_include1 !=-1: 
        pos_circles=[]
        pos_circle=ch.find('•',pos_include1+1)
        pos_dot=ch.find(".", pos_include1)
        while pos_circle!=-1 and pos_circle<pos_dot:
            pos_circles.append(pos_circle)
            pos_circle=ch.find("•",pos_circle+1)
        for i in range(len(pos_circles)-1):   
            final.append(ch[:pos_include1+8]+" "+ch[pos_circles[i]+2:pos_circles[i+1]-1])    
        final.append(ch[:pos_include1+8]+" "+ch[pos_circles[-1]+2:ch.find(".",pos_circles[-1])])
        return ". ".join(final)
    return ch

In [561]:
data['content'] = data['content'].apply(lambda ch:suchAsCircle(ch))

In [562]:
data[data['subconcept']=='Plan_Risk_Management_Expert_Judgment']['content'][6]

'expert_judgment considered from groups_or_individuals such As Senior_Management. expert_judgment considered from groups_or_individuals such As Project_Stakeholders. expert_judgment considered from groups_or_individuals such As Project_Managers who have worked on projects in the same area( directly or through lessonslearned). expert_judgment considered from groups_or_individuals such As Subject_Matter_Experts( smes) in business or Project_Area. expert_judgment considered from groups_or_individuals such As Industry_Groups and consultants, an. expert_judgment considered from groups_or_individuals such As professional and technical associations'

In [563]:
data2 = data.copy()

In [415]:
data = data2.copy()

In [564]:
data['content'][3]

'the Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.\n'

In [565]:
def includeCircledot(ch,title):
    final=[]
    pos_include1=ch.find(' include:')
    if pos_include1 !=-1: 
        pos_circles=[]
        circle_sentences=[]
        pos_white_spaces = []
        blck_circle_pos = pos_include1
        while True:
            blck_circle_pos = ch.find('•',blck_circle_pos+1)
            if blck_circle_pos==-1:
                break
            else:
                white_space_pos = ch.find('. ',blck_circle_pos)
                slash_n_pos = ch.find('.\n',blck_circle_pos)
                if white_space_pos!=-1 and (slash_n_pos==-1 or (slash_n_pos!=-1 and slash_n_pos>white_space_pos)):
                    pos_circles.append(blck_circle_pos)
                    pos_white_spaces.append(white_space_pos)
        if len(pos_circles)!=0:
            pos_include2 = ch.find(' include:',pos_include1+1)
            if pos_include2!=-1:
                pos_include1 = pos_include2
            posdot=0
            while posdot<pos_include1:
                if ch.find('.',posdot+1)==-1 or ch.find('.',posdot+1)>pos_include1:
                    break
                posdot=ch.find('.',posdot+1)
            final.append(ch[:posdot]+'.\n')
            for i in range(len(pos_circles)): 
                indiv = ch[pos_circles[i]+2:pos_white_spaces[i]].split()
                indiv = '_'.join([ch[0].upper()+ch[1:] for ch in indiv])
                final.append(title+' include'+' '+indiv) 
#               final.append(ch[:pos_include1+8]+" "+indiv) 
            for i in range(len(pos_circles)-1):
                circle_sent=ch[pos_circles[i]+2:pos_circles[i+1]-1].replace('. ',' ')
                final.append(circle_sent)
            final.append(ch[pos_circles[-1]+2:].replace('. ',' '))
            return ". ".join(final)
    return ch

In [566]:
for i in data.index:
    data['content'][i] = includeCircledot(data['content'][i], data['subconcept'][i])

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [567]:
data['content'][58]

'.\n. Plan_Risk_Responses_Project_Management_Plan_Updates include Schedule_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Cost_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Quality_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Procurement_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Human_Resource_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Scope_Baseline. Plan_Risk_Responses_Project_Management_Plan_Updates include Schedule_Baseline. Plan_Risk_Responses_Project_Management_Plan_Updates include Cost_Baseline. Schedule_Management_Plan the Schedule_Management_Plan is updated to reflect changes in process and practice driven by the Risk_Responses this may include changes in tolerance or behavior related to Resource_Loading and leveling, as well as updates to the Schedule_Strategy.. Cost_Management_Plan the Cost_Management_Plan is update

In [568]:
for i in data.index:
    text = data['content'][i]
    l = text.split(' ')
    for x in range(len(l)):
        if len(l[x])>0:
            if l[x][0].islower() and '_' in l[x]:
                l[x] = '_'.join([ch[0].upper()+ch[1:] for ch in l[x].split('_')])
    text = ' '.join(l)
    data['content'][i] = text

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [569]:
data['content'][6]

'Expert_Judgment considered from Groups_Or_Individuals such As Senior_Management. Expert_Judgment considered from Groups_Or_Individuals such As Project_Stakeholders. Expert_Judgment considered from Groups_Or_Individuals such As Project_Managers who have worked on projects in the same area( directly or through lessonslearned). Expert_Judgment considered from Groups_Or_Individuals such As Subject_Matter_Experts( smes) in business or Project_Area. Expert_Judgment considered from Groups_Or_Individuals such As Industry_Groups and consultants, an. Expert_Judgment considered from Groups_Or_Individuals such As professional and technical associations'

In [103]:
# def includeCirclecomma(ch,title):
#     final=[]
#     pos_include1=ch.find(' include:')
#     if pos_include1 !=-1: 
#         pos_circles=[]
#         blck_circle_pos = pos_include1
#         while True:
#             blck_circle_pos = df1.find('•',blck_circle_pos+1)
#             if blck_circle_pos==-1:
#                 break
#             else:
#                 white_space_pos = df1.find('. ',blck_circle_pos)
#                 slash_n_pos = df1.find('.\n',blck_circle_pos)
#                 if not (white_space_pos!=-1 and (slash_n_pos==-1 or (slash_n_pos!=-1 and slash_n_pos>white_space_pos))):
#                     pos_circles.append(blck_circle_pos)
#         pos_circle=ch.find('•',pos_include1+1)
#         pos_dot=ch.find(".", pos_include1)
#         while pos_circle!=-1 and pos_circle<pos_dot:
#             pos_circles.append(pos_circle)
#             pos_circle=ch.find("•",pos_circle+1)
#         posdot=0
#         while posdot<pos_include1:
#             if ch.find('.',posdot+1)==-1 or ch.find('.',posdot+1)>pos_include1:
#                 break
#             posdot=ch.find('.',posdot+1)
#         final.append(ch[:posdot]+'.')
#         for i in range(len(pos_circles)-1):
#             indiv = ch[pos_circles[i]+2:pos_circles[i+1]-1]
#             final.append(title+' include'+' '+indiv)
#         final.append(title+' include'+' '+ch[pos_circles[-1]+2:ch.find(".",pos_circles[-1])])
#         final.append(ch[ch.find(".",pos_circles[-1])+1:])
#         return ". ".join(final)
#     return ch

In [570]:
def includeCirclecomma(ch,title):
    final=[]
    pos_include1=ch.find(' include:')
    if pos_include1 !=-1: 
        pos_circles=[]
        pos_circle=ch.find('•',pos_include1+1)
        pos_dot=ch.find(".", pos_include1)
        while pos_circle!=-1 and pos_circle<pos_dot:
            pos_circles.append(pos_circle)
            pos_circle=ch.find("•",pos_circle+1)
        posdot=0
        while posdot<pos_include1:
            if ch.find('.',posdot+1)==-1 or ch.find('.',posdot+1)>pos_include1:
                break
            posdot=ch.find('.',posdot+1)
        final.append(ch[:posdot]+'.')
        for i in range(len(pos_circles)-1):   
            indiv = ch[pos_circles[i]+2:pos_circles[i+1]-1]
            final.append(title+' include'+' '+indiv)
        final.append(title+' include'+' '+ch[pos_circles[-1]+2:pos_dot])
        final.append(ch[pos_dot+1:])
        return ". ".join(final)
    return ch

In [571]:
for i in data.index:
    data['content'][i] = includeCirclecomma(data['content'][i], data['subconcept'][i])

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [572]:
data['content'][58]

'.\n. Plan_Risk_Responses_Project_Management_Plan_Updates include Schedule_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Cost_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Quality_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Procurement_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Human_Resource_Management_Plan. Plan_Risk_Responses_Project_Management_Plan_Updates include Scope_Baseline. Plan_Risk_Responses_Project_Management_Plan_Updates include Schedule_Baseline. Plan_Risk_Responses_Project_Management_Plan_Updates include Cost_Baseline. Schedule_Management_Plan the Schedule_Management_Plan is updated to reflect changes in process and practice driven by the Risk_Responses this may include changes in tolerance or behavior related to Resource_Loading and leveling, as well as updates to the Schedule_Strategy.. Cost_Management_Plan the Cost_Management_Plan is update

In [107]:
# pos_circles=[]
# pos_white_spaces = []
# blck_circle_pos = pos_include1
# while True:
#     blck_circle_pos = df1.find('•',blck_circle_pos+1)
#     if blck_circle_pos==-1:
#         break
#     else:
#         white_space_pos = df1.find('. ',blck_circle_pos)
#         slash_n_pos = df1.find('.\n',blck_circle_pos)
#         if white_space_pos!=-1 and (slash_n_pos==-1 or (slash_n_pos!=-1 and slash_n_pos>white_space_pos)):
#             pos_circles.append(blck_circle_pos)
#             pos_white_spaces.append(white_space_pos)

In [108]:
# blck_circle_pos = 0
# while True:
#     blck_circle_pos = df1.find('•',blck_circle_pos+1)
#     if blck_circle_pos==-1:
#         break
#     else:
#         white_space_pos = df1.find('. ',blck_circle_pos)
#         slash_n_pos = df1.find('.\n',blck_circle_pos)
#         if white_space_pos!=-1 and (slash_n_pos==-1 or (slash_n_pos!=-1 and slash_n_pos>white_space_pos)):
#             df1 = df1[:white_space_pos] + ' ' + df1[white_space_pos+2:]

In [573]:
def suchAsAnd(ch):
    ch = ch.replace(', .','.')
    ch = ch.replace(',.','.')
    ch = ch.replace(', and.','.')
    ch = ch.replace('  ',' ')
    print(ch,'\n\n\n')
    sp = spacy.load('en_core_web_sm')
    doc = sp(ch)
    index_suchas=None
    new_chs=[]
    for word in range(len(doc)-2):
        if doc[word].text.lower()=='such' and doc[word+1].text.lower()=='as' and doc[word-1].text!='(':
            index_suchas=word+2
            break
    if index_suchas:
        suchas_elements=[]
        if doc[index_suchas].dep_=='det':
            index_suchas+=1
        if doc[index_suchas].dep_=='ADP' and doc[index_suchas+1].dep_=='det':
            index_suchas+=2
        if doc[index_suchas].dep_=='pobj':
            print('7ajaa')
            suchas_elements.append(doc[index_suchas].text)
            last_element_end=index_suchas+1
        for i in range(index_suchas+1,len(doc)-1):
            if (doc[i].dep_ not in ['conj','cc','det','appos']) and doc[i].text!=',':
                break
            elif doc[i].dep_ in ['conj','appos']:
                suchas_elements.append(doc[i].text)
                last_element_end=i+1
        pre=doc[0].text
        try:
            post=doc[last_element_end].text
        except:
            return ch
        for i in range(1,len(doc)):
            if(i<=index_suchas-1):
                pre+=" "+doc[i].text
            if(i>last_element_end):
                post+=" "+doc[i].text
        for k in range(len(suchas_elements)):
            new_chs.append(pre+" "+suchas_elements[k]+" "+post)
        return ". ".join(new_chs)
    return ch

In [110]:
# def nounOfNoun(ch):
#     sp = spacy.load('en_core_web_sm')
#     doc = sp(ch)
#     index_of=None
#     for word in range(len(doc)-2):
#         if doc[word].text.lower()=='of':
#             index_of=word
#             break
#     if index_of:
#         of_left_dep_index=None
#         pcomp_right_dep_index=None
#         of_right_dep_index=None
#         pcomp_right_dep=[]
#         of_left_dep = [t.text for t in doc if doc[index_of] in t.rights][0]
#         of_right_dep = [t.text for t in doc[index_of].rights][0]
#         print(of_left_dep," ",of_right_dep)

#         for i in range(index_of,-1,-1):
#             if doc[i].text==of_left_dep:
#                 of_left_dep_index=i
#                 print([t.text for t in doc[of_left_dep_index].lefts])
#                 prep_left_dep = [t.text for t in doc[of_left_dep_index].lefts][0]
#             if of_left_dep_index and doc[i].text== prep_left_dep:
#                 prep_left_dep_index=i
#                 break
                
                
                
#         for i in range(index_of,len(doc)):
#             if doc[i].dep_=="pobj" and doc[i].text==of_right_dep:
#                 of_right_dep_index =i
#                 pcomp_right_dep_index=i
#                 break
#             if doc[i].dep_=="pcomp" and doc[i].text==of_right_dep:
#                 of_right_dep_index =i
#                 pcomp_right_dep_index=i
#                 pcomp_right_dep = [t.text for t in doc[of_right_dep_index].rights][0]
#             if pcomp_right_dep and doc[i].dep_=="dobj" and doc[i].text==pcomp_right_dep:
#                 pcomp_right_dep_index=i
#                 break
#         pre=doc[:prep_left_dep_index+1]
#         new_first=doc[index_of+1:pcomp_right_dep_index+1]
#         new_last=doc[prep_left_dep_index+1:index_of]
#         post=doc[pcomp_right_dep_index+1:]
#         new_ch=""
#         for i in range(len(pre)):
#             new_ch+=" "+pre[i].text
#         for i in range(len(new_first)):
#             new_ch+=" "+new_first[i].text
#         for i in range(len(new_last)):
#             new_ch+=" "+new_last[i].text
#         for i in range(len(post)):
#             new_ch+=" "+post[i].text
#         return new_ch
#     return ch

In [111]:
# for i in data.index:
#     start=0
#     while True:
#         if data['content'][i].lower().find(' of ')!=-1:
#             print(data['content'][i])
#             data['content'][i] = nounOfNoun(data['content'][i].lower())
#         elif data['content'][i].lower().find(' of ')==-1:
#             break

In [574]:
data['content'] = data['content'].apply(lambda ch:suchAsAnd(ch))

in planning Risk_Management, all approved Subsidiary_Management_Plans and baselines should be taken into consideration in order to make the Risk_Management_Plan consistent with them. the Risk_Management_Plan is also a component of the Plan_Risk_Management_Project_Management_Plan. the Plan_Risk_Management_Project_Management_Plan provides baseline or current state of Risk_Affected_Areas including scope, schedule, and cost.
 



the Project_Charter can provide various Plan_Risk_Management_Inputs such as High_Level_Risks, High_Level Project_Descriptions, and High_Level_Requirements.
 



7ajaa
the Plan_Risk_Management_Stakeholder_Register, which contains all details related to the Project_Stakeholders, provides an overview of their roles.
 



the Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.
 



.. Plan_Risk_M

key elements of the Identify_Risks_Risk_Management_Plan that contribute to the identify Risks_Process are the assignments of roles and responsibilities, provision for Risk_Management_Activities in the budget and schedule, and categories of risk, which are sometimes expressed as a Risk_Breakdown_Structure( figure 11 - 4).
 



the Identify_Risks_Cost_Management_Plan provides processes and controls that can be used to help identify risks across the project.
 



the Identify_Risks_Schedule_Management_Plan provides insight to project time / Schedule_Objectives and expectations which may be impacted by risks( known and unknown).
 



the Quality_Management_Plan provides a baseline of Quality_Measures and metrics for use in identifying risks.
 



the Human_Resource_Management_Plan provides guidance on how Project_Human_Resources should be defined, staffed, managed, and eventually released. it can also contain roles and responsibilities, Project_Organization_Charts, and the Staffing_Managem

risks may be identified directly by experts with relevant experience with similar projects or Business_Areas. such experts should be identified by the Project_Manager and invited to consider all aspects of the project and suggest possible risks based on their previous experience and areas of expertise. the experts ’ bias should be taken into account in this process.
 



the primary Identify_Risks_Outputs from identify risks is the initial entry into the Identify_Risks_Risk_Register. the Identify_Risks_Risk_Register is a document in which the results of Risk_Analysis and Risk_Response_Planning are recorded. it contains the outcomes of the other Risk_Management_Processes as they are conducted, resulting in an increase in the level and type of information contained in the Identify_Risks_Risk_Register over time. the preparation of the Identify_Risks_Risk_Register begins in the identify Risks_Process with the following information, and then is available to other Project_Management and Risk

risks requiring Near_Term_Responses may be considered more urgent to address. indicators of priority may include probability of detecting the risk, time to affect a Risk_Response, symptoms and Warning_Signs, and the Risk_Rating. in some qualitative analyses, the assessment of Risk_Urgency is combined with the Risk_Ranking that is determined from the probability and Impact_Matrix to give a final Risk_Severity_Rating.
 



Perform_Qualitative_Risk_Analysis_Expert_Judgment is required to assess the probability and impact of each risk to determine its location in the matrix shown in figure 11 - 10. experts generally are those having experience with similar, recent projects. gathering Perform_Qualitative_Risk_Analysis_Expert_Judgment is often accomplished with the use of Risk_Facilitation_Workshops or interviews. the experts ’ bias should be taken into account in this process.
 



.
. Perform_Qualitative_Risk_Analysis_Project_Documents_Updates include Risk_Register_Updates. Perform_Qualita

7ajaa
Project_Documents are updated with information resulting from Quantitative_Risk_Analysis.
. Perform_Quantitative_Risk_Analysis_Project_Documents_Updates include Probabilistic_Analysis_Of_The_Project. Perform_Quantitative_Risk_Analysis_Project_Documents_Updates include Probability_Of_Achieving_Cost_And_Time_Objectives. Perform_Quantitative_Risk_Analysis_Project_Documents_Updates include Prioritized_List_Of_Quantified_Risks. Perform_Quantitative_Risk_Analysis_Project_Documents_Updates include Trends_In_Quantitative_Risk_Analysis_Results. Probabilistic_Analysis of the project estimates are made of Potential_Project_Schedule and cost outcomes listing the possible Completion_Dates and costs with their associated Confidence_Levels this output, often expressed as a cumulative frequency distribution, is used with Stakeholder_Risk_Tolerances to permit quantification of the cost and Time_Contingency_Reserves such Contingency_Reserves are needed to bring the risk of overrunning stated Proje

some responses are designed for use only if certain events occur. for some risks, it is appropriate for the Project_Team to make a Response_Plan that will only be executed under certain predefined conditions, if it is believed that there will be sufficient warning to implement the plan. events that trigger the Contingency_Response such as missing intermediate milestones or gaining higher priority with a supplier, should be defined and tracked. Risk_Responses identified using this Plan_Risk_Responses_Tools_And_Techniques are often called Contingency_Plans or Fallback_Plans and include identified triggering events that set the plans in effect.
 



Plan_Risk_Responses_Expert_Judgment is Inputs from knowledgeable parties pertaining to the actions to be taken on a specific and defined risk. expertise may be provided by any group or person with specialized education, knowledge, skill, experience, or training in establishing Risk_Responses.
 



.
. Plan_Risk_Responses_Project_Management_Pla

many Control_Processes employ Variance_Analysis to compare the planned results to the Actual_Results. for the purposes of controlling risks, trends in the Project_Execution should be reviewed using Performance_Information. earned Value_Analysis and other methods of Project_Variance_And_Trend_Analysis may be used for monitoring overall Project_Performance. outcomes from these analyses may forecast Potential_Deviation of the project at completion from cost and Schedule_Targets. deviation from the Baseline_Plan may indicate the Potential_Impact of threats or opportunities.
 



Technical_Performance_Measurement compares technical accomplishments during Project_Execution to the schedule of technical achievement. it requires the definition of objective, quantifiable measures of technical performance, which can be used to compare Actual_Results against targets. such technical Performance_Measures may include weight, Transaction_Times, number of delivered defects, Storage_Capacity, etc. devia

In [575]:
data['content'][3]

'the Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.\n'

In [576]:
data.to_csv('data/subconcepts.csv')

In [115]:
# ents = pd.DataFrame(columns=('named entity', 'output'))
# for i in data.index:
#     d = []
#     text = data['content'][i]
#     doc = sp(text)
#     for ent in doc.ents:
#         d.append((ent.label_, ent.text))
#     df_entity = pd.DataFrame(d, columns=('named entity', 'output'))
#     ents = pd.concat([ents,df_entity])
# ents = ents.drop_duplicates().reset_index(drop=True)
# ents.head(20)

In [577]:
datacont=data['content']

In [578]:
content=" "
for d in datacont:
    content= content+d

In [579]:
content

' in planning Risk_Management, all approved Subsidiary_Management_Plans and baselines should be taken into consideration in order to make the Risk_Management_Plan consistent with them. the Risk_Management_Plan is also a component of the Plan_Risk_Management_Project_Management_Plan. the Plan_Risk_Management_Project_Management_Plan provides baseline or current state of Risk_Affected_Areas including scope, schedule, and cost.\nthe Project_Charter can provide various Plan_Risk_Management_Inputs such as High_Level_Risks , High_Level Project_Descriptions , and High_Level_Requirements . \nthe Plan_Risk_Management_Stakeholder_Register, which contains all details related to the Project_Stakeholders, provides an overview of their roles.\nthe Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.\n.. Plan_Risk_Management_Organi

In [580]:
alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [581]:
L= []
for i in range(len(content)):
    if content[i].upper() not in alpha:
        if content[i] not in L:
            L.append(content[i])
L

[' ',
 '_',
 ',',
 '.',
 '\n',
 '(',
 ')',
 '’',
 '1',
 '-',
 '4',
 '0',
 '"',
 '“',
 '”',
 '/',
 '7',
 '2',
 '5',
 ':',
 '•',
 '3',
 '6',
 '—',
 '$',
 '%',
 ';']

In [582]:
useful = ['_', ',', '.', '?', '1', '4', '0', '7', '2', '5', '3', '6', '$', '%']

In [583]:
content=content[1:]

In [584]:
content[:1000]

'in planning Risk_Management, all approved Subsidiary_Management_Plans and baselines should be taken into consideration in order to make the Risk_Management_Plan consistent with them. the Risk_Management_Plan is also a component of the Plan_Risk_Management_Project_Management_Plan. the Plan_Risk_Management_Project_Management_Plan provides baseline or current state of Risk_Affected_Areas including scope, schedule, and cost.\nthe Project_Charter can provide various Plan_Risk_Management_Inputs such as High_Level_Risks , High_Level Project_Descriptions , and High_Level_Requirements . \nthe Plan_Risk_Management_Stakeholder_Register, which contains all details related to the Project_Stakeholders, provides an overview of their roles.\nthe Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand.\n.. Plan_Risk_Management_Organiz

In [585]:
data['content'] = data['content'].apply(lambda ch:ch.replace('High_Level Project_Descriptions', 'High_Level_Project_Descriptions'))

In [586]:
# content = content.replace('High_Level Project_Descriptions', 'High_Level_Project_Descriptions')

In [587]:
def remove_ponc_num(data,ponc):
    text = ' '
    for i in range(len(data)):
        if not (data[i] in ponc) or data[i] in useful:
            text+=data[i]
        else:
            if text[-1] not in useful and data[i+1] not in ponc:
                text+=' '
    return text
# content = remove_ponc_num(content, L)[1:]
# content[:1000]

In [588]:
data['content'] = data['content'].apply(lambda ch:remove_ponc_num(ch,L))

In [589]:
contentseg = re.split(r"[.?]", content)
contentseg

['in planning Risk_Management, all approved Subsidiary_Management_Plans and baselines should be taken into consideration in order to make the Risk_Management_Plan consistent with them',
 ' the Risk_Management_Plan is also a component of the Plan_Risk_Management_Project_Management_Plan',
 ' the Plan_Risk_Management_Project_Management_Plan provides baseline or current state of Risk_Affected_Areas including scope, schedule, and cost',
 '\nthe Project_Charter can provide various Plan_Risk_Management_Inputs such as High_Level_Risks , High_Level Project_Descriptions , and High_Level_Requirements ',
 ' \nthe Plan_Risk_Management_Stakeholder_Register, which contains all details related to the Project_Stakeholders, provides an overview of their roles',
 '\nthe Plan_Risk_Management_Enterprise_Environmental_Factors that can influence the Plan_Risk_Management_Process include Risk_Attitudes, thresholds, and tolerances that describe the degree of risk that an organization will withstand',
 '\n',
 ''

In [590]:
sentences = [re.split(r"[.?]", content) for content in data['content']]

In [591]:
sentences = [[ch for ch in l if ch not in ['', ' ']] for l in sentences]

In [592]:
data['sentences'] = sentences

In [593]:
def remove_spaces(text,alpha):
    final_output = ''
    for i in range(len(text)):
        if text[i].upper() in alpha or text[i] in useful:
            final_output+=text[i]
            if i!=len(text)-1:
                if text[i+1]==' ' and i+1!=len(text)-1:
                    final_output+=' '
        elif text[i]==' ':
                continue 
    return final_output
# new_content = []
# for z in range(len(contentseg)):
#     new_content.append(remove_spaces(contentseg[z],alpha))

In [594]:
sentences = [[remove_spaces(ch,alpha) for ch in l ] for l in data['sentences']]

In [595]:
data['sentences'] = sentences

In [ ]:
# new_content

In [ ]:
# new_content1_2 = []
# for senten in range(len(new_content)):
#     words = []
#     words = [x for x in new_content[senten].split()]
#     for i in words:
#         if len(i)<=4 and i not in new_content1_2:
#             new_content1_2.append(i)
# new_content1_2

In [ ]:
# bt1 = []
# for x in new_content:
#     if len(x)<=10 and x not in bt1:
#         bt1.append(x)
# bt1

In [ ]:
# new_content

In [ ]:
# new_content = [x.lower() for x in new_content if x not in ['','figure', 'section']]

In [ ]:
# new_content

In [596]:
sentences = [[ch.lower() for ch in l if ch not in ['','figure', 'section']] for l in data['sentences']]

In [597]:
data['sentences'] = sentences

In [ ]:
# to_df = pd.DataFrame(new_content, columns=['sentences'])
# # to_df.head()

In [598]:
for i in data.index:
    text_list = data['sentences'][i]
    for t in range(len(text_list)):
        l = text_list[t].split(' ')
        for x in range(len(l)):
            if len(l[x])>0:
                if l[x][0].islower() and '_' in l[x]:
                    l2 = l[x].split('_')
                    for d in l2:
                        if len(d)<=1:
                            print(d)
                    l[x] = '_'.join([ch[0].upper()+ch[1:] for ch in l[x].split('_') if len(ch)>1]) 
        text = ' '.join(l)
        text_list[t] = text
    data['sentences'][i] = text_list


2
2
2
2
y
x


C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
# for i in to_df.index:
#     text = to_df['sentences'][i]
#     l = text.split(' ')
#     for x in range(len(l)):
#         if len(l[x])>0:
#             if l[x][0].islower() and '_' in l[x]:
#                 l2 = l[x].split('_')
#                 for d in l2:
#                     if len(d)<=1:
#                         print(d)
#                 l[x] = '_'.join([ch[0].upper()+ch[1:] for ch in l[x].split('_') if len(ch)>1]) 
#     text = ' '.join(l)
#     to_df['sentences'][i] = text

In [ ]:
# to_df['sentences'][0]

In [599]:
data['sentences'][0]

['in planning Risk_Management,all approved Subsidiary_Management_Plans and baselines should be taken into consideration in order to make the Risk_Management_Plan consistent with them',
 'the Risk_Management_Plan is also a component of the Plan_Risk_Management_Project_Management_Plan',
 'the Plan_Risk_Management_Project_Management_Plan provides baseline or current state of Risk_Affected_Areas including scope,schedule,and cost']

In [ ]:
# for i in to_df['sentences']:
#     doc = sp(i)
#     print(doc.text)
#     L = [token for token in doc.text]
    

In [ ]:
# for i in to_df.index:
#     if to_df['sentences'][i].find(' such as')!=-1:
#         print(str(to_df['sentences'][i].find(' is also a'))+'\t'+'the index of df is: '+str(i))

In [ ]:
# to_df['sentences'][5]

In [600]:
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
stop_words = set(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meddhafer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [601]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [602]:
to_delete=["have","has","by","with","in","on","off","up","down","about", "after", "against", "are", "as", "be", "between", "during", "for", "from", "into", "of", "to", "under"]
stop_words.difference_update(to_delete)

In [603]:
import spacy
sp = spacy.load('en_core_web_sm')
L = [word.text for word in sp(''.join(stopwords)) if word.pos_ in ['ADV']]

In [604]:
L

[]

In [605]:
sp('part')[0].pos_

'NOUN'

In [606]:
def pre_process(text):
    import spacy
    sp = spacy.load('en_core_web_sm')
    char_to_replace = {' such as ': ' such.as ', ' is a ': ' is.a ', ' is an ': ' is.an ', ' is part of ': ' is.part.of '," is known as ":" is.known.as "}
    
    # remove ADV
    text = ' '.join([word.text for word in sp(text) if word.pos_ !='ADV'])
    # Iterate over all key-value pairs in dictionary 
    for key, value in char_to_replace.items():
        # Replace key character with value character in string
        text = text.replace(key, value)
    ##Convert to list from string
    text = text.split()
    
    text = ' '.join([word for word in text if word not in stop_words])
    text = text.replace('.',' ')
    text = ' '.join([word.text for word in sp(text) if word.pos_ !='ADJ' or word.text=='such'])
    return text


In [ ]:
# to_df['sentences'] = to_df['sentences'].apply(lambda x:pre_process(x))

In [607]:
sentences = [[pre_process(ch) for ch in l ] for l in data['sentences']]

In [608]:
data['sentences'] = sentences

In [609]:
data['sentences'][0]

['in planning Risk_Management , approved Subsidiary_Management_Plans baselines be taken into consideration in order to make with',
 'Risk_Management_Plan is a component of Plan_Risk_Management_Project_Management_Plan',
 'Plan_Risk_Management_Project_Management_Plan provides state of Risk_Affected_Areas including scope , schedule , cost']

**POS tagging :**

In [ ]:
# pre_pos_tag_list = [to_df['sentences'][i] for i in to_df.index]
# pre_pos_tag_list[:6]

In [ ]:
# pre_pos_tag = ' '.join(pre_pos_tag_list)

In [ ]:
# pre_pos_tag[:1000]

In [ ]:
# import spacy
# sp = spacy.load('en_core_web_sm')
# # create a spaCy document that we will be using to perform parts of speech tagging.
# doc = sp(pre_pos_tag)
# print(doc.text)

In [ ]:
# print(doc[6].pos_) #include
# print(doc[6].tag_)
# print(spacy.explain(doc[6].tag_))
# print(doc[6].text)

In [ ]:
# num = doc.count_by(spacy.attrs.POS)
# for k,v in sorted(num.items()):
#     print(f'{k}. {doc.vocab[k].text:{8}}: {v}')

In [ ]:
# print(doc.ents)
# for entity in doc.ents:
#     print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

In [ ]:
# from collections import Counter
# complete_doc = sp(pre_pos_tag)
# words = [token.text for token in complete_doc if  not token.is_punct]
# word_freq = Counter(words)
# # 100 commonly occurring words with their frequencies
# common_words = word_freq.most_common(100)
# print (common_words)

In [ ]:
# unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
# print (unique_words[:10])
# print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"]) 

In [ ]:
# doc = sp(pre_pos_tag)
# spacy.displacy.render(doc, style='dep', jupyter=True, options={'distance': 200})

In [ ]:
# !pip install wordcloud
# from wordcloud import WordCloud, STOPWORDS
# word_cloud = WordCloud(
#                           background_color='white',
#                           stopwords=stop_words,
#                           max_words=100,
#                           max_font_size=50, 
#                           random_state=42
#                          ).generate(str(pd.Series(pre_pos_tag.split())))
# print(word_cloud)
# fig = plt.figure(1)
# plt.imshow(word_cloud)
# plt.axis('off')
# plt.show()
# fig.savefig("word.png", dpi=900)

In [ ]:
# ents = pd.DataFrame(columns=('named entity', 'output'))
# for i in to_df.index:
#     doc = sp(to_df['sentences'][i])
#     d = []
#     for ent in doc.ents:
#         d.append((ent.label_, ent.text))
#     df_entity = pd.DataFrame(d, columns=('named entity', 'output'))
#     ents = pd.concat([ents,df_entity],ignore_index=True)
# ents.head(20)

In [ ]:
# ents.to_csv("data/entities.csv")

**Lemmatization:**

In [ ]:
# lemmatizer = WordNetLemmatizer()
# def lemm(s):
#     l = []
#     for word,tag in pos_tag(word_tokenize(s)):
#         wntag = tag [0].lower()
#         wntag =wntag if wntag in ['a','r','n','v'] else None
#         lemma = lemmatizer.lemmatize(word,wntag) if wntag else word
#         l.append(lemma)
#     return l

In [610]:
lemmatizer = WordNetLemmatizer()
def lemm2(s):
    l = []
    for word,tag in pos_tag(word_tokenize(s)):
        wntag = tag [0].lower()
        wntag =wntag if wntag in ['a','r','n','v'] else None
        lemma = lemmatizer.lemmatize(word,wntag) if wntag else word
        l.append(lemma)
    return ' '.join(l)

In [ ]:
# to_df['sentences'] = to_df['sentences'].apply(lambda s : " ".join(lemm(s)))

In [ ]:
# to_df['sentences'][0]

In [611]:
sentences = [[lemm2(ch) for ch in l ] for l in data['sentences']]

In [612]:
data['sentences'] = sentences

In [613]:
data['sentences'][0]

['in plan Risk_Management , approve Subsidiary_Management_Plans baseline be take into consideration in order to make with',
 'Risk_Management_Plan be a component of Plan_Risk_Management_Project_Management_Plan',
 'Plan_Risk_Management_Project_Management_Plan provide state of Risk_Affected_Areas include scope , schedule , cost']

**Synonyms:**

In [ ]:
# def syns(s):
#     l = []
#     for w in sp(s):
#         syn = wordnet.synsets(w.text)
#         a = w.text
#         if len(syn) > 0 and w.pos_ in ['NN','VB']:
#             a = syn[0].name().split('.')[0]
#         l.append(a)
#     return l

In [614]:
def syns2(s):
    l = []
    for w in sp(s):
        syn = wordnet.synsets(w.text)
        a = w.text
        if len(syn) > 0 and w.pos_ in ['NN','VB']:
            a = syn[0].name().split('.')[0]
        l.append(a)
    return ' '.join(l)

In [ ]:
# to_df['sentences'] = to_df['sentences'].apply(lambda s : " ".join(syns(s)))

In [ ]:
# to_df['sentences'][13]

In [615]:
sentences = [[syns2(ch) for ch in l ] for l in data['sentences']]

In [616]:
data['sentences'] = sentences

In [617]:
data['sentences'][0]

['in plan Risk_Management , approve Subsidiary_Management_Plans baseline be take into consideration in order to make with',
 'Risk_Management_Plan be a component of Plan_Risk_Management_Project_Management_Plan',
 'Plan_Risk_Management_Project_Management_Plan provide state of Risk_Affected_Areas include scope , schedule , cost']

**Tokenization:**

In [ ]:
# tokens = [word_tokenize(sent) for sent in to_df.sentences]

In [ ]:
# to_df['Tokens'] = tokens
# to_df.head()

In [ ]:
# to_df['sentences'][0]

In [618]:
sentences = [[word_tokenize(ch) for ch in l ] for l in data['sentences']]

In [619]:
data['Tokens'] = sentences

In [620]:
data['Tokens'][0]

[['in',
  'plan',
  'Risk_Management',
  ',',
  'approve',
  'Subsidiary_Management_Plans',
  'baseline',
  'be',
  'take',
  'into',
  'consideration',
  'in',
  'order',
  'to',
  'make',
  'with'],
 ['Risk_Management_Plan',
  'be',
  'a',
  'component',
  'of',
  'Plan_Risk_Management_Project_Management_Plan'],
 ['Plan_Risk_Management_Project_Management_Plan',
  'provide',
  'state',
  'of',
  'Risk_Affected_Areas',
  'include',
  'scope',
  ',',
  'schedule',
  ',',
  'cost']]

**Named Entity Extraction: :** 

In [ ]:
# d = []
# pre_named_entity_list = [to_df['sentences'][i] for i in to_df.index]
# pre_named = ' '.join(pre_named_entity_list)
# pre_named_doc = sp(pre_named)

In [ ]:
# for ent in pre_named_doc.ents:
#     d.append((ent.label_, ent.text))
#     df_entity = pd.DataFrame(d, columns=('named entity', 'output'))
# df_entity.head()

In [ ]:
# df_entity.loc[df_entity['named entity'] == 'ORG'].tail()

**Evaluating the Retrieval Process: TF-IDF measures:**

In [ ]:
# !pip install gensim
# from gensim.corpora import Dictionary
# processed_docs = to_df.Tokens
# dct = Dictionary(processed_docs)
# print(dct)

In [ ]:
# final_processed_docs = [[word for word in l]for l in to_df.Tokens]

In [ ]:
# final_processed_docs

In [ ]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
# document = Dictionary(final_processed_docs)
# print(document)

In [ ]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
# doc_term_matrix = [document.doc2bow(doc) for doc in to_df.Tokens]

In [ ]:
# !pip install gensim
# import gensim
# Creating the object for LDA model using gensim library
# Lda = gensim.models.ldamodel.LdaModel

In [ ]:
# Running and Trainign LDA model on the document term matrix.
# ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = document, passes=4)

# print(ldamodel.print_topics(num_topics=2, num_words=3))

In [ ]:
# L = []
# for i in to_df.index:
#     ch = ' such as ' in to_df['sentences'][i]
#     if ch==False:
#         L.append(ch)
#     else:
#         print(i)
#         print(to_df['sentences'][i] + '\n\n')
# print(len(L))
# to_df.shape

In [ ]:
# testdf=to_df.copy()

In [ ]:
# testdf

In [621]:
test_data=data.copy()

In [622]:
test_data

,concept,type,process_desc,subconcept,content,infos,sentences,Tokens
0,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management,all approved Subs...",{},"[in plan Risk_Management , approve Subsidiary_...","[[in, plan, Risk_Management, ,, approve, Subsi..."
1,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various Plan_...,{0: {'about': 'plan risk management project ch...,[Project_Charter provide Plan_Risk_Management_...,"[[Project_Charter, provide, Plan_Risk_Manageme..."
2,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,the Plan_Risk_Management_Stakeholder_Register...,{0: {'about': 'plan risk management stakeholde...,"[Plan_Risk_Management_Stakeholder_Register , c...","[[Plan_Risk_Management_Stakeholder_Register, ,..."
3,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Plan_Risk_Management_Enterprise_Environme...,{0: {'about': 'plan risk management enterprise...,[Plan_Risk_Management_Enterprise_Environmental...,[[Plan_Risk_Management_Enterprise_Environmenta...
4,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,..Plan_Risk_Management_Organizational_Process...,{0: {'about': 'plan risk management organizati...,[Plan_Risk_Management_Organizational_Process_A...,[[Plan_Risk_Management_Organizational_Process_...
...,...,...,...,...,...,...,...,...
70,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Work_Performance_Information,"Work_Performance_Information,as a Control_Ris...",{},"[Work_Performance_Information , as Control_Ris...","[[Work_Performance_Information, ,, as, Control..."
71,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Change_Requests,implementing Contingency_Plans or workarounds...,{},[implement Contingency_Plans workarounds resul...,"[[implement, Contingency_Plans, workarounds, r..."
72,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Management_Plan_Updates,if the approved Change_Requests have an effec...,{},[approve Change_Requests have effect on Risk_M...,"[[approve, Change_Requests, have, effect, on, ..."
73,Control_Risks,Control_Risks_Outputs,control risks is the process of implementing r...,Control_Risks_Project_Documents_Updates,Project_Documents that may be updated as a re...,{},[Project_Documents may be update as result of ...,"[[Project_Documents, may, be, update, as, resu..."


**Spacy Rule Based Matching:**

In [ ]:
# doc=sp(to_df['sentences'][0])

In [623]:
def last_child(token):
    rights = token.rights
    child = None
    for txt in rights:
        child = txt
        break
    return child

In [ ]:
# def learn_ontologies (doc):
#     subjects = [] 
#     objects = [] 
#     links = []
#     ontologies = [] 
#     # Extracting subjects
#     for token in doc: 
#         if token.dep_ in ['nsubj']:
#             ontology = {'subject': '-', 'link': [], 'object': '-'} 
#             ontology['subject'] = token 
#             ontologies.append(ontology)
#     # Extracting Links (verbs) 
#     for i in range (len(ontologies)):
#         ontologies[i]['link'].append(ontologies[i]['subject'].head)
#         if last_child (ontologies[i]['link'][-1]):
#             if last_child (ontologies[i]['link'][-1]).dep_ in ['attr', 'prep', 'acomp']:
#                 print(last_child(ontologies[i]['link'][-1]))
#                 ontologies[i]['link'].append(last_child(ontologies[i]['link'][-1])) 
#             elif last_child(ontologies[i]['link'][-1]).dep in ['case']:
#                 ontologies[i]['link'].append(ontologies[i]['link'][-1].head)
#     # Extracting objects
#     for i in range (len (ontologies)):
#         if last_child (ontologies[i]['link'][-1]): 
#             if last_child(ontologies[i]['link'][-1]).dep_ in ['pobj', 'dobj']:
#                 ontologies[i]['object'] = last_child(ontologies[i]['link' ][-1]) 
#             elif last_child(ontologies[i]['link' ][-1]).dep in ['poss']:
#                 child = last_child(ontologies[i]['link'][-1])
#                 ontologies[i]['object'] = child 
#             else:
#                 child = last_child(ontologies[i]['link'][-1]) 
#                 ontologies[i]['object'] = last_child(child)
#     return ontologies

In [ ]:
# to_df['sentences'] = to_df['sentences'].apply(lambda x: learn_ontologies(sp(x)))

In [ ]:
# to_df['sentences'][0]

In [ ]:
# listtest=[]
# for i in range(len(to_df['sentences'])):
#     for j in to_df['sentences'][i]:
#         listtest.append(j)

In [624]:
def learn_ontologies (doc,title):
    subjects = [] 
    objects = [] 
    links = []
    Concept = []
    ontologies = [] 
    # Extracting subjects
    for token in doc: 
        if token.dep_ in ['nsubj']:
            ontology = {'concept':title,'subject': '-', 'link': [], 'object': '-'} 
            ontology['subject'] = token 
            ontologies.append(ontology)
    # Extracting Links (verbs) 
    for i in range (len(ontologies)):
        ontologies[i]['link'].append(ontologies[i]['subject'].head)
        if last_child (ontologies[i]['link'][-1]):
            if last_child (ontologies[i]['link'][-1]).dep_ in ['attr', 'prep', 'acomp']:
                print(last_child(ontologies[i]['link'][-1]))
                ontologies[i]['link'].append(last_child(ontologies[i]['link'][-1])) 
            elif last_child(ontologies[i]['link'][-1]).dep in ['case']:
                ontologies[i]['link'].append(ontologies[i]['link'][-1].head)
    # Extracting objects
    for i in range (len (ontologies)):
        if last_child (ontologies[i]['link'][-1]): 
            if last_child(ontologies[i]['link'][-1]).dep_ in ['pobj', 'dobj']:
                ontologies[i]['object'] = last_child(ontologies[i]['link' ][-1]) 
            elif last_child(ontologies[i]['link' ][-1]).dep in ['poss']:
                child = last_child(ontologies[i]['link'][-1])
                ontologies[i]['object'] = child 
            else:
                child = last_child(ontologies[i]['link'][-1]) 
                ontologies[i]['object'] = last_child(child)
    return ontologies

In [625]:
sentences = [[learn_ontologies(sp(ch),data['type'][i]) for ch in  data['sentences'][i]] for i in data.index]

component
use
from
from
from
from
from
from
define
develop
assign
by
tailor
summarize
component
use
use
grid
grid
revise
record
audit
express
use
of
in
in
in
during
interview
Identify_Risks_Inputs
with
indicator
part
part
way
way
use
use
on
prune
review
know
for
Graphical_Representations
with
from
from
document
record
cause
tailor
assess
assess
accord
assess
assess
accord
assess
assess
accord
assess
assess
accord
on
assign
use
shade
tailor
in
monitor
Perform_Qualitative_Risk_Analysis_Tools_And_Techniques
to
as
with
experience
with
as
on
upon
in
component
use
show
with
value
on
type
express
use
update
make
need
estimate
as
to
to
with
use
strategy
strategy
use
act
example
action
use
select
select
design
update
update
update
update
update
update
write
address
for
in
during
of
to
in
in
schedule
frequency
conduct
review
with
to
for
in
plan
performance
revise
update
update
update


In [626]:
data['sentences'] = sentences

In [627]:
data['sentences'][0]

[[],
 [{'concept': 'Plan_Risk_Management_Inputs',
   'subject': Risk_Management_Plan,
   'link': [be, component],
   'object': Plan_Risk_Management_Project_Management_Plan}],
 [{'concept': 'Plan_Risk_Management_Inputs',
   'subject': Plan_Risk_Management_Project_Management_Plan,
   'link': [provide],
   'object': state}]]

In [628]:
listtest=[]
for i in range(len(data['sentences'])):
    for j in data['sentences'][i]:
        for x in j:
            listtest.append(x)

In [629]:
relation=pd.DataFrame(listtest)

In [630]:
relation

,concept,subject,link,object
0,Plan_Risk_Management_Inputs,Risk_Management_Plan,"[be, component]",Plan_Risk_Management_Project_Management_Plan
1,Plan_Risk_Management_Inputs,Plan_Risk_Management_Project_Management_Plan,[provide],state
2,Plan_Risk_Management_Inputs,Project_Charter,[provide],Plan_Risk_Management_Inputs
3,Plan_Risk_Management_Inputs,Plan_Risk_Management_Stakeholder_Register,[contain],relate
4,Plan_Risk_Management_Inputs,Plan_Risk_Management_Enterprise_Environmental_...,[influence],None
...,...,...,...,...
387,Control_Risks_Outputs,Risk_Management_Processes,[produce],information
388,Control_Risks_Outputs,Organizational_Process_Assets_Updates,[include],template
389,Control_Risks_Outputs,Organizational_Process_Assets_Updates,[include],Risk_Breakdown_Structure
390,Control_Risks_Outputs,Organizational_Process_Assets_Updates,[include],learn


In [631]:
relation=relation.dropna()

In [632]:
for index,row in relation.iterrows():
    if isinstance(row['object'],str):
        relation.drop(index,inplace=True)

C:\Users\meddhafer\anaconda3\envs\deep\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [633]:
relation.reset_index(inplace=True,drop=True)

In [634]:
for index,row in relation.iterrows():
    if str(row['object'])=="figure":
        relation.drop(index,inplace=True)
relation.reset_index(inplace=True,drop=True)

In [635]:
relation.shape

(333, 4)

In [636]:
relation

,concept,subject,link,object
0,Plan_Risk_Management_Inputs,Risk_Management_Plan,"[be, component]",Plan_Risk_Management_Project_Management_Plan
1,Plan_Risk_Management_Inputs,Plan_Risk_Management_Project_Management_Plan,[provide],state
2,Plan_Risk_Management_Inputs,Project_Charter,[provide],Plan_Risk_Management_Inputs
3,Plan_Risk_Management_Inputs,Plan_Risk_Management_Stakeholder_Register,[contain],relate
4,Plan_Risk_Management_Inputs,influence,[include],Risk_Attitudes
...,...,...,...,...
328,Control_Risks_Outputs,information,[help],Project_Managers
329,Control_Risks_Outputs,Risk_Management_Processes,[produce],information
330,Control_Risks_Outputs,Organizational_Process_Assets_Updates,[include],template
331,Control_Risks_Outputs,Organizational_Process_Assets_Updates,[include],Risk_Breakdown_Structure


In [637]:
relation.to_csv("data/relation.csv")


In [638]:
# #define the pattern 
# pattern = [{'DEP':'compound', 'OP':'?'},
#           {'POS':'NOUN'},
#           {'POS':'VERB'},
#           {'DEP':'compound', 'OP':'?'},
#           {'POS':'NOUN'}]
# Pattern is a
# pattern1 = [{'POS':'NOUN', 'op':'?'}, 
#             {'POS':'PROPN', 'op':'?'},
#             {'LOWER': 'is'},  {'LOWER': 'a'}, 
#             {'POS': 'NOUN', 'op':'?'},
#             {'POS':'PROPN', 'op':'?'}]

# #patter is an
# pattern2 = [{'POS':'NOUN', 'op':'?'}, 
#             {'POS':'PROPN', 'op':'?'},
#             {'LOWER': 'is'},  {'LOWER': 'an'}, 
#             {'POS':'NOUN', 'op':'?'}, 
#             {'POS':'PROPN', 'op':'?'}]
# Pattern such as
pattern3 = [{'POS':'NOUN', 'op':'?'}, 
            {'POS':'PROPN', 'op':'?'}, 
            {'LOWER': 'such'},  {'LOWER': 'as'},
            {'POS':'NOUN', 'op':'?'}, 
            {'POS':'PROPN', 'op':'?'}]
# Pattern is part of
pattern4 = [{'POS':'NOUN', 'op':'?'}, 
            {'POS':'PROPN', 'op':'?'}, 
            {'LOWER': 'is'},{'LOWER': 'part'},  {'LOWER': 'of'}, 
            {'POS':'NOUN', 'op':'?'}, 
            {'POS':'PROPN', 'op':'?'}]

In [639]:
sp('define')[0].tag_

'VB'

In [640]:
# content = content.replace('figure','')

In [332]:
# to_df['sentences']

0                                                     []
1      [{'subject': Risk_Management_Plan, 'link': [be...
2      [{'subject': Plan_Risk_Management_Project_Mana...
3      [{'subject': Project_Charter, 'link': [provide...
4      [{'subject': Plan_Risk_Management_Stakeholder_...
                             ...                        
382    [{'subject': Organizational_Process_Assets_Upd...
383    [{'subject': Organizational_Process_Assets_Upd...
384    [{'subject': Organizational_Process_Assets_Upd...
385    [{'subject': document, 'link': [be, update], '...
386                                                   []
Name: sentences, Length: 387, dtype: object

In [337]:
# df = testdf['sentences'][:]

In [338]:
# df

0      in plan Risk_Management , approve Subsidiary_M...
1      Risk_Management_Plan be a component of Plan_Ri...
2      Plan_Risk_Management_Project_Management_Plan p...
3      Project_Charter provide Plan_Risk_Management_I...
4      Plan_Risk_Management_Stakeholder_Register , co...
                             ...                        
382    Organizational_Process_Assets_Updates include ...
383    Organizational_Process_Assets_Updates include ...
384    Organizational_Process_Assets_Updates include ...
385         document be update as needed Project_Closure
386    version of Risk_Register Risk_Management_Plan_...
Name: sentences, Length: 387, dtype: object

In [641]:
test_data.head()

,concept,type,process_desc,subconcept,content,infos,sentences,Tokens
0,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Project_Management_Plan,"in planning Risk_Management,all approved Subs...",{},"[in plan Risk_Management , approve Subsidiary_...","[[in, plan, Risk_Management, ,, approve, Subsi..."
1,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Project_Charter,the Project_Charter can provide various Plan_...,{0: {'about': 'plan risk management project ch...,[Project_Charter provide Plan_Risk_Management_...,"[[Project_Charter, provide, Plan_Risk_Manageme..."
2,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Stakeholder_Register,the Plan_Risk_Management_Stakeholder_Register...,{0: {'about': 'plan risk management stakeholde...,"[Plan_Risk_Management_Stakeholder_Register , c...","[[Plan_Risk_Management_Stakeholder_Register, ,..."
3,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Enterprise_Environmental_...,the Plan_Risk_Management_Enterprise_Environme...,{0: {'about': 'plan risk management enterprise...,[Plan_Risk_Management_Enterprise_Environmental...,[[Plan_Risk_Management_Enterprise_Environmenta...
4,Plan_Risk_Management,Plan_Risk_Management_Inputs,plan risk management is the process of definin...,Plan_Risk_Management_Organizational_Process_As...,..Plan_Risk_Management_Organizational_Process...,{0: {'about': 'plan risk management organizati...,[Plan_Risk_Management_Organizational_Process_A...,[[Plan_Risk_Management_Organizational_Process_...


In [642]:
sentences = [[ch.replace('be a ', 'is a ') for ch in l ] for l in test_data['sentences']]

In [643]:
test_data['sentences'] = sentences

In [644]:
test = test_data['sentences'][:]

In [335]:
# spans1 = []
# matcher = Matcher(sp(content).vocab, validate=True) 
# matcher.add(key="All match",on_match=None,patterns=[pattern1,pattern2,pattern3,pattern4,pattern5])
# matchess = matcher(sp(content))
# spans1.append([sp(content)[start:end] for _, start, end in matchess])

In [339]:
# spans1 = []
# for i in df:
#     matcher = Matcher(sp(i).vocab, validate=True) 
#     matcher.add(key="All match",on_match=None,patterns=[pattern1,pattern2,pattern3,pattern4])
#     matchess = matcher(sp(i))
#     spans1.append([sp(i)[start:end] for _, start, end in matchess])

In [645]:
spans1 = []
for l in test:
    for i in l:
        matcher = Matcher(sp(i).vocab, validate=True) 
        matcher.add(key="All match",on_match=None,patterns=[pattern3,pattern4])
        matchess = matcher(sp(i))
        spans1.append([sp(i)[start:end] for _, start, end in matchess])

In [646]:
spans1

[[],
 [],
 [],
 [Plan_Risk_Management_Inputs such as,
  such as,
  Plan_Risk_Management_Inputs such as High_Level_Risks,
  such as High_Level_Risks],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [technique such as, such as, technique such as use, such as use],
 [],
 [Groups_Or_Individuals such as,
  such as,
  Groups_Or_Individuals such as Senior_Management,
  such as Senior_Management],
 [Groups_Or_Individuals such as,
  such as,
  Groups_Or_Individuals such as Project_Stakeholders,
  such as Project_Stakeholders],
 [Groups_Or_Individuals such as,
  such as,
  Groups_Or_Individuals such as Project_Managers,
  such as Project_Managers],
 [Groups_Or_Individuals such as, such as],
 [Groups_Or_Individuals such as,
  such as,
  Groups_Or_Individuals such as Industry_Groups,
  such as Industry_Groups],
 [Groups_Or_Individuals such as,
  such as,
  Groups_Or_Individuals such as association,
  such as association],
 [],
 [],
 [],
 [],
 [],
 [],
 [term such as, such as, term su

In [647]:
L = []
span1_2 = []
for ind in range(len(spans1)):
    L = spans1[ind]
    l2 = L[:]
    for l in range(len(L)):
        for senten in range(len(L)):
            if L[senten]!=L[l]:
                if str(L[senten]).find(str(L[l]))!=-1:
                    l2.remove(L[l])
                    break
    span1_2.append(l2)

In [648]:
span1_3 = [[l for l in L] for L in span1_2 if len(L)>0]
span1_4 = []
for i in range(len(span1_3)):
    for j in range(len(span1_3[i])):
        span1_4.append(span1_3[i][j])

In [649]:
span1_4

[Plan_Risk_Management_Inputs such as High_Level_Risks,
 technique such as use,
 Groups_Or_Individuals such as Senior_Management,
 Groups_Or_Individuals such as Project_Stakeholders,
 Groups_Or_Individuals such as Project_Managers,
 Groups_Or_Individuals such as,
 Groups_Or_Individuals such as Industry_Groups,
 Groups_Or_Individuals such as association,
 term such as level,
 risk such as,
 such as Industry_Studies,
 such as Risk_Databases,
 Project_Objective such as schedule,
 Project_Objective such as cost,
 Project_Objective such as quality,
 Project_Objective such as performance,
 objective such as cost,
 risk such as use,
 category such as Project_Phase,
 such as Industry_Studies,
 such as Risk_Databases,
 value such as duration,
 event such as outcome,
 such as,
 Input_Values such as,
 histogram such as cost,
 input such as Probability_Distributions,
 trigger Contingency_Response such as miss,
 strategy such as alteration,
 deviation such as]

In [643]:
# ch = 'risk activities recorded benefit project'
# l1 = ch.split()
# l = [sp(text)[0].pos_=='VERB' for text in l1]
# for i in range(len(l)):
#     if l[i]==True:
#         ind = i
# reg = str(re.escape(l1[ind]))
# start = re.search(reg,ch).start()
# end = ch.find(' ',start)
# print(ch[start:end])

benefit


In [82]:
# def verb_relation(text):
#     doc = sp(text)
#     l = [doc[x].pos_=='VERB' for x in range(len(text.split()))]
#     rel = 'rien'
#     for i in range(len(l)):
#         if l[i]==True:
#             ind = i
#             reg = str(re.escape(text.split()[ind]))
#             start = re.search(reg,text).start()
#             end = text.find(' ',start)
#             rel =text[start:end]
#             break
#     return re.split('( %s )' % rel,text)

In [83]:
# verb_relation('probability impact defined project context')

['probability impact', ' defined ', 'project context']

In [84]:
# verb_relation('management plan provides state')

['management plan', ' provides ', 'state']

In [646]:
# sp('plan')[0].pos_

'VERB'

In [647]:
# len(span1_4)

90

In [650]:
for span in span1_4:
    print(span.text)

Plan_Risk_Management_Inputs such as High_Level_Risks
technique such as use
Groups_Or_Individuals such as Senior_Management
Groups_Or_Individuals such as Project_Stakeholders
Groups_Or_Individuals such as Project_Managers
Groups_Or_Individuals such as
Groups_Or_Individuals such as Industry_Groups
Groups_Or_Individuals such as association
term such as level
risk such as
such as Industry_Studies
such as Risk_Databases
Project_Objective such as schedule
Project_Objective such as cost
Project_Objective such as quality
Project_Objective such as performance
objective such as cost
risk such as use
category such as Project_Phase
such as Industry_Studies
such as Risk_Databases
value such as duration
event such as outcome
such as
Input_Values such as
histogram such as cost
input such as Probability_Distributions
trigger Contingency_Response such as miss
strategy such as alteration
deviation such as


In [651]:
X_Relation_Y=[]
for span in span1_4:
    print(span.text)
    X_Relation_Y.append(re.split('( such as )',span.text))
    X_Relation_Y.append(re.split('( is an )',span.text))
    X_Relation_Y.append(re.split('( is a )',span.text))
    X_Relation_Y.append(re.split('( is part of )',span.text))
    
   # 'is a','such as','kind of','including','have','part','member','instance','and or other','especially','like','in'
df_X_Relation_Y = pd.DataFrame(X_Relation_Y,columns=['Y','Relation','X'])
df_X_Relation_Y.head(20)

Plan_Risk_Management_Inputs such as High_Level_Risks
technique such as use
Groups_Or_Individuals such as Senior_Management
Groups_Or_Individuals such as Project_Stakeholders
Groups_Or_Individuals such as Project_Managers
Groups_Or_Individuals such as
Groups_Or_Individuals such as Industry_Groups
Groups_Or_Individuals such as association
term such as level
risk such as
such as Industry_Studies
such as Risk_Databases
Project_Objective such as schedule
Project_Objective such as cost
Project_Objective such as quality
Project_Objective such as performance
objective such as cost
risk such as use
category such as Project_Phase
such as Industry_Studies
such as Risk_Databases
value such as duration
event such as outcome
such as
Input_Values such as
histogram such as cost
input such as Probability_Distributions
trigger Contingency_Response such as miss
strategy such as alteration
deviation such as


,Y,Relation,X
0,Plan_Risk_Management_Inputs,such as,High_Level_Risks
1,Plan_Risk_Management_Inputs such as High_Level...,None,None
2,Plan_Risk_Management_Inputs such as High_Level...,None,None
3,Plan_Risk_Management_Inputs such as High_Level...,None,None
4,technique,such as,use
5,technique such as use,None,None
6,technique such as use,None,None
7,technique such as use,None,None
8,Groups_Or_Individuals,such as,Senior_Management
9,Groups_Or_Individuals such as Senior_Management,None,None


In [652]:
df_ind2 = df_X_Relation_Y.drop_duplicates().reset_index(drop=True).rename(columns={'X':'individual','Y':'concept'})
for i in df_ind2.index:
    if df_ind2['Relation'][i]==None:
        df_ind2.drop(index=i,axis=0,inplace=True)
df_ind2 = df_ind2.reset_index(drop=True)
df_ind2.head(20)

,concept,Relation,individual
0,Plan_Risk_Management_Inputs,such as,High_Level_Risks
1,technique,such as,use
2,Groups_Or_Individuals,such as,Senior_Management
3,Groups_Or_Individuals,such as,Project_Stakeholders
4,Groups_Or_Individuals,such as,Project_Managers
5,Groups_Or_Individuals,such as,Industry_Groups
6,Groups_Or_Individuals,such as,association
7,term,such as,level
8,Project_Objective,such as,schedule
9,Project_Objective,such as,cost


In [653]:
df_ind2.shape

(21, 3)

In [654]:
df_ind2.to_csv("data/suchasrelation.csv")

In [655]:
df_ind3 = df_ind2[['concept']].drop_duplicates().reset_index(drop=True)
df_ind3

,concept
0,Plan_Risk_Management_Inputs
1,technique
2,Groups_Or_Individuals
3,term
4,Project_Objective
5,objective
6,risk
7,category
8,value
9,event


In [656]:
annotations

,about,description
0,Project_Charter,described in section 4.1.3.1.
1,Plan_Risk_Management_Stakeholder_Register,described in section 13.1.3.1.
2,Plan_Risk_Management_Enterprise_Environmental_...,described in section 2.1.5.
3,Plan_Risk_Management_Organizational_Process_As...,described in section 2.1.4.
4,Definitions_Of_Risk_Probability_And_Impact,table 11-1 is an example of definitions of ne...
5,Identify_Risks_Risk_Management_Plan,described in section 11.1.3.1.
6,Identify_Risks_Cost_Management_Plan,described in section 7.1.3.1.
7,Identify_Risks_Schedule_Management_Plan,described in section 6.1.3.1.
8,Quality_Management_Plan,described in section 8.1.3.1.
9,Human_Resource_Management_Plan,described in section 9.1.3.1.


In [657]:
annotations['about'] = annotations['about'].apply(lambda ch: '_'.join([word[0].upper()+word[1:]for word in ch.split(' ') if word!='']))

In [658]:
annotations

,about,description
0,Project_Charter,described in section 4.1.3.1.
1,Plan_Risk_Management_Stakeholder_Register,described in section 13.1.3.1.
2,Plan_Risk_Management_Enterprise_Environmental_...,described in section 2.1.5.
3,Plan_Risk_Management_Organizational_Process_As...,described in section 2.1.4.
4,Definitions_Of_Risk_Probability_And_Impact,table 11-1 is an example of definitions of ne...
5,Identify_Risks_Risk_Management_Plan,described in section 11.1.3.1.
6,Identify_Risks_Cost_Management_Plan,described in section 7.1.3.1.
7,Identify_Risks_Schedule_Management_Plan,described in section 6.1.3.1.
8,Quality_Management_Plan,described in section 8.1.3.1.
9,Human_Resource_Management_Plan,described in section 9.1.3.1.


In [659]:
annotations.to_csv('annotations.csv')